In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import pickle
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
pd.options.mode.chained_assignment = None  # default='warn'
import re
import time

# Tickers

## S&P 500

In [2]:
sp_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp_response = requests.get(sp_url)
print(sp_response.status_code)

200


In [3]:
sp_soup = BeautifulSoup(sp_response.text, 'html.parser')
sp_table = sp_soup.find('table',{'class':"wikitable"})

In [4]:
sp_df = pd.read_html(str(sp_table))
sp_df = pd.DataFrame(sp_df[0])
sp_df['Symbol'] = sp_df['Symbol'].str.replace('.','-',regex = True)
sp_df = sp_df.set_index('Symbol')
sp_df

,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
Symbol,,,,,,,,
MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
...,...,...,...,...,...,...,...,...
YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


## NASDAQ 100

In [5]:
daq_url = 'https://en.wikipedia.org/wiki/Nasdaq-100'
daq_response = requests.get(daq_url)
print(daq_response.status_code)

200


In [6]:
daq_soup = BeautifulSoup(daq_response.text, 'html.parser')
daq_table = daq_soup.find_all('table',{'class':"wikitable"})

In [7]:
daq_df = pd.read_html(str(daq_table))
daq_df = pd.DataFrame(daq_df[3])
daq_df.rename({'Ticker': 'Symbol', 'Company':'Security'}, axis = 'columns', inplace = True)
daq_df['Symbol'] = daq_df['Symbol'].str.replace('.','-',regex = True)
daq_df = daq_df.set_index('Symbol')
daq_df

,Security,GICS Sector,GICS Sub-Industry
Symbol,,,
ATVI,Activision Blizzard,Communication Services,Interactive Home Entertainment
ADBE,Adobe Inc.,Information Technology,Application Software
ADP,ADP,Information Technology,Data Processing & Outsourced Services
ABNB,Airbnb,Consumer Discretionary,Internet & Direct Marketing Retail
ALGN,Align Technology,Health Care,Health Care Supplies
...,...,...,...
WBA,Walgreens Boots Alliance,Consumer Staples,Drug Retail
WDAY,"Workday, Inc.",Information Technology,Application Software
XEL,Xcel Energy,Utilities,Multi-Utilities


## TSX 60

In [8]:
tsx_url = 'https://en.wikipedia.org/wiki/S%26P/TSX_60'
tsx_response = requests.get(tsx_url)
print(tsx_response.status_code)

200


In [9]:
tsx_soup = BeautifulSoup(tsx_response.text, 'html5lib')
tsx_table = tsx_soup.find_all('table',{'class':"wikitable"})

In [10]:
tsx_df = pd.read_html(str(tsx_table))
tsx_df = pd.DataFrame(tsx_df[0])
tsx_df['Symbol'] = tsx_df['Symbol'].str.replace('.','-',regex = True)
tsx_df.replace(np.nan, 'NA', inplace=True)
for i, ticker in enumerate(tsx_df.Symbol):
    tsx_df.Symbol[i] = ticker + '.TO'
tsx_df = tsx_df.set_index('Symbol')
tsx_df.rename({'Company':'Security'}, axis = 'columns', inplace = True)
tsx_df

,Security,Sector
Symbol,,
AEM.TO,Agnico Eagle Mines Limited,Basic Materials
AQN.TO,Algonquin Power & Utilities Corp.,Utilities
ATD.TO,Alimentation Couche-Tard Inc.,Consumer Staples
BCE.TO,BCE Inc.,Communication Services
BMO.TO,Bank of Montreal,Financial Services
BNS.TO,Bank of Nova Scotia,Financial Services
ABX.TO,Barrick Gold Corporation,Basic Materials
BHC.TO,Bausch Health Companies Inc.,Healthcare
BAM-A.TO,Brookfield Asset Management Inc.,Financial Services


## MSCI Mexico

In [11]:
mex_df = pd.read_csv("https://www.ishares.com/us/products/239670/ishares-msci-mexico-capped-etf/1467271812596.ajax?fileType=csv&fileName=EWW_holdings&dataType=fund", skiprows = 9)
mex_df.dropna(inplace = True)
mex_df = mex_df[mex_df['Sector'] != 'Cash and/or Derivatives']
mex_df.rename({'Ticker':'Symbol', 'Name':'Security'}, axis = 'columns', inplace = True)
mex_df['Symbol'] = mex_df['Symbol'].str.replace('*','',regex = True)
for i, ticker in enumerate(mex_df.Symbol):
    mex_df.Symbol[i] = ticker + '.MX'
mex_df = mex_df.set_index('Symbol')
mex_df = mex_df[['Security']]
mex_df

,Security
Symbol,
AMXL.MX,AMERICA MOVIL L
WALMEX.MX,WALMART DE MEXICO V
GFNORTEO.MX,GPO FINANCE BANORTE
FEMSAUBD.MX,FOMENTO ECONOMICO MEXICANO
CEMEXCPO.MX,CEMEX CPO
GMEXICOB.MX,GRUPO MEXICO B
BIMBOA.MX,GRUPO BIMBO A
GAPB.MX,GRUPO AEROPORTUARIO DEL PACIFICO
GFINBURO.MX,GRUPO FINANCIERO INBURSA SRIES O


## ASX 200

In [12]:
asx_url = 'https://en.wikipedia.org/wiki/S%26P/ASX_200'
asx_response = requests.get(asx_url)
print(asx_response.status_code)

200


In [13]:
asx_soup = BeautifulSoup(asx_response.text, 'html.parser')
asx_table = asx_soup.find('table',{'class':"wikitable"})

In [14]:
asx_df = pd.read_html(str(asx_table))
asx_df = pd.DataFrame(asx_df[0])
asx_df.rename({'Code':'Symbol', 'Company':'Security'}, axis = 'columns', inplace = True)
asx_df['Symbol'] = asx_df['Symbol'].str.replace('.','-',regex = True)
for i, ticker in enumerate(asx_df.Symbol):
    asx_df.Symbol[i] = ticker + '.AX'
asx_df = asx_df.set_index('Symbol')
asx_df

,Security,Sector,Market Capitalisation,Chairperson,HQ
Symbol,,,,,
A2M.AX,The a2 Milk Company,Consumer Staples,4.222573e+09,David Hearn,Auckland
ABC.AX,Adbri,Materials,2.114513e+09,Raymond Barro,Sydney
ABP.AX,Abacus Property Group,Real Estate,2.504890e+09,Myra Salkinder,Sydney
AGL.AX,AGL Energy,Utilities,5.361206e+09,Peter Botten,Sydney
AIA.AX,Auckland Airport,Industrials,1.042566e+10,Dr Patrick Strange,Auckland
...,...,...,...,...,...
WDS.AX,Woodside Energy,Energy,3.393550e+10,Richard Goyder,Perth
WPR.AX,Waypoint REIT[15],Real Estate,1.756310e+09,NaN,NaN
WTC.AX,Wisetech Global,Information Technology,5.798960e+09,Andrew Harrison,Sydney


## OBX

In [15]:
obx_url = 'https://en.wikipedia.org/wiki/OBX_Index'
obx_response = requests.get(obx_url)
print(obx_response.status_code)

200


In [16]:
obx_soup = BeautifulSoup(obx_response.text, 'html.parser')
obx_table = obx_soup.find_all('table',{'class':'wikitable'})

In [17]:
obx_df = pd.read_html(str(obx_table))
obx_df = pd.DataFrame(obx_df[0])
obx_df.rename({'Ticker symbol':'Symbol', 'Company':'Security'}, axis = 'columns', inplace = True)
for i, ticker in enumerate(obx_df.Symbol):
    obx_df.Symbol[i] = ticker + '.OL'
obx_df = obx_df.set_index('Symbol')
obx_df = obx_df[['Security', 'GICS Sector']]
obx_df

,Security,GICS Sector
Symbol,,
AKER.OL,Aker,multi-sector holdings
AKERBP.OL,Aker BP,oil and gas exploration and production
BAKKA.OL,Bakkafrost,packaged foods and meats
BWLPG.OL,BW LPG,oil and gas storage and transportation
BWO.OL,BW Offshore,oil and gas equipment and services
DNB.OL,DNB,diversified banks
DNO.OL,DNO,oil and gas exploration and production
ENTRA.OL,Entra,real estate operating companies
EQNR.OL,Equinor,integrated oil and gas


In [18]:
# df.dropna(axis = 1, how = 'all')

## DAX

In [19]:
dax_url = "https://en.wikipedia.org/wiki/DAX"
dax_response = requests.get(dax_url)
print(dax_response.status_code)

200


In [20]:
dax_soup = BeautifulSoup(dax_response.text, 'html.parser')
dax_table = dax_soup.find_all('table',{'class':"wikitable"})

In [21]:
dax_df = pd.read_html(str(dax_table))
dax_df = pd.DataFrame(dax_df[2])
dax_df = dax_df.drop('Unnamed: 0', axis = 1)
dax_df.rename({'Ticker symbol':'Symbol', 'Company':'Security'}, axis = 'columns', inplace = True)
dax_df = dax_df.set_index('Symbol')
dax_df

,Security,Prime Standard Sector,Index weighting (%)1,Employees,Founded
Symbol,,,,,
ADS.DE,Adidas,Textile,2.5,"061,401 (2021)",1924
AIR.DE,Airbus,Aerospace,4.8,126.495 (2021),1970
ALV.DE,Allianz,Financial Services,6.8,"155,411 (2021)",1890
BAS.DE,BASF,Basic Materials,3.8,"111,047 (2021)",1865
BAYN.DE,Bayer,Healthcare,5.5,"099,637 (2021)",1863
BEI.DE,Beiersdorf,Consumer goods,NaN,"020,567 (2021)",1882
BMW.DE,BMW,Automobile,2.3,"118,909 (2021)",1916
BNR.DE,Brenntag,Distribution,0.9,"017,200 (2021)",1874
CON.DE,Continental,Automotive,0.7,"236,386 (2020)",1871


## AEX

In [22]:
aex_url = "https://en.wikipedia.org/wiki/AEX_index"
aex_response = requests.get(aex_url)
print(aex_response.status_code)

200


In [23]:
aex_soup = BeautifulSoup(aex_response.text, 'html.parser')
aex_table = aex_soup.find_all('table',{'class':'wikitable'})

In [24]:
aex_df = pd.read_html(str(aex_table))
aex_df = pd.DataFrame(aex_df[1])
aex_df.rename({'Ticker symbol':'Symbol', 'Company':'Security'}, axis = 'columns', inplace = True)
for i, ticker in enumerate(aex_df.Symbol):
    aex_df.Symbol[i] = ticker + '.AS'
aex_df = aex_df.set_index('Symbol')
aex_df

,Security,ICB Sector,Index weighting (%)
Symbol,,,
ADYEN.AS,Adyen,"support services, financial administration",6.90
AGN.AS,Aegon,life insurance,0.81
AD.AS,Ahold Delhaize,food retailers & wholesalers,3.40
AKZA.AS,AkzoNobel,specialty chemicals,2.58
MT.AS,ArcelorMittal,iron & steel,2.41
ASM.AS,ASM International,semiconductors,1.70
ASML.AS,ASML Holding,semiconductors,17.62
UMG.AS,Universal Music Group,entertainment,0.57
BESI.AS,BE Semiconductors,semiconductors,0.66


## CAC 40

In [25]:
cac_url = "https://en.wikipedia.org/wiki/CAC_40"
cac_response = requests.get(cac_url)
print(cac_response.status_code)

200


In [26]:
cac_soup = BeautifulSoup(cac_response.text, 'html.parser')
cac_table = cac_soup.find_all('table',{'class':"wikitable"})

In [27]:
cac_df = pd.read_html(str(cac_table))
cac_df = pd.DataFrame(cac_df[2])
cac_df.rename({'Ticker':'Symbol', 'Company':'Security'}, axis = 'columns', inplace = True)
cac_df = cac_df.set_index('Symbol')
cac_df

,Security,Sector,GICS Sub-Industry
Symbol,,,
AI.PA,Air Liquide,Basic Materials,Industrial Gases
AIR.PA,Airbus,Industrials,Aerospace & Defense
ALO.PA,Alstom,Industrials,Rail Transport
MT.AS,ArcelorMittal,Basic Materials,Steel
CS.PA,AXA,Financial Services,Life & Health Insurance
BNP.PA,BNP Paribas,Financial Services,Diversified Banks
EN.PA,Bouygues,Industrials,Construction & Engineering
CAP.PA,Capgemini,Technology,IT Consulting & Other Services
CA.PA,Carrefour,Consumer Defensive,Hypermarkets & Super Centers


## FTSE 100

In [28]:
ftse_url = 'https://en.wikipedia.org/wiki/FTSE_100_Index'
ftse_response = requests.get(ftse_url)
print(ftse_response.status_code)

200


In [29]:
ftse_soup = BeautifulSoup(ftse_response.text, 'html.parser')
ftse_table = ftse_soup.find_all('table',{'class':"wikitable"})

In [30]:
ftse_df = pd.read_html(str(ftse_table))
ftse_df = pd.DataFrame(ftse_df[2])
ftse_df.drop('FTSE Industry Classification Benchmark sector[14]', axis = 1, inplace = True)
ftse_df.rename({'EPIC':'Symbol', 'Company':'Security'}, axis = 'columns', inplace = True)
for i, ticker in enumerate(ftse_df.Symbol):
    ftse_df.Symbol[i] = ticker + '.L'
ftse_df = ftse_df.set_index('Symbol')
ftse_df

,Security
Symbol,
III.L,3i
ABDN.L,Abrdn
ADM.L,Admiral Group
AAF.L,Airtel Africa
AAL.L,Anglo American plc
...,...
UU.L,United Utilities
UTG.L,Unite Group
VOD.L,Vodafone Group


## FTSE MIB

In [31]:
mib_url = "https://en.wikipedia.org/wiki/FTSE_MIB"
mib_response = requests.get(mib_url)
print(mib_response.status_code)

200


In [32]:
mib_soup = BeautifulSoup(mib_response.text, 'html.parser')
mib_table = mib_soup.find_all('table',{'class':"wikitable"})

In [33]:
mib_df = pd.read_html(str(mib_table))
mib_df = pd.DataFrame(mib_df[1])
mib_df.rename({'Ticker':'Symbol', 'Company':'Security'}, axis = 'columns', inplace = True)
mib_df = mib_df.set_index('Symbol')
mib_df

,Security,ISIN (and link to quote webpage),ICB Sector
Symbol,,,
A2A.MI,A2A,IT0001233417,7530 (Electricity)[citation needed]
AMP.MI,Amplifon,IT0004056880,(Health Care)
ATL.MI,Atlantia,IT0003506190,2770 (Industrial Transportation)[citation needed]
AZM.MI,Azimut,IT0003261697,8770 (Financial Services)[citation needed]
BGN.MI,Banca Generali,IT0001031084,8770 (Financial Services)
BMED.MI,Banca Mediolanum,IT0004776628,8770 (Financial Services)
BAMI.MI,Banco BPM,IT0005218380,8350 (Banks)
BPE.MI,BPER Banca,IT0000066123,8350 (Banks)[citation needed]
CPR.MI,Campari,IT0003849244,3530 (Beverages)[citation needed]


## IBEX 35

In [34]:
ibex_url = "https://en.wikipedia.org/wiki/IBEX_35"
ibex_response = requests.get(ibex_url)
print(ibex_response.status_code)

200


In [35]:
ibex_soup = BeautifulSoup(ibex_response.text, 'html.parser')
ibex_table = ibex_soup.find_all('table',{'class':"wikitable"})

In [36]:
ibex_df = pd.read_html(str(ibex_table))
ibex_df = pd.DataFrame(ibex_df[0])
ibex_df.rename({'Ticker':'Symbol', 'Company':'Security'}, axis = 'columns', inplace = True)
for i, ticker in enumerate(ibex_df.Symbol):
    ibex_df.Symbol[i] = ticker + '.MC'
ibex_df = ibex_df.set_index('Symbol')
ibex_df

,Security,Headquarters,Sector,ISIN,Weighting
Symbol,,,,,
ANA.MC,Acciona,"Alcobendas, Madrid",Construction,ES0125220311,0.99
ACX.MC,Acerinox,Madrid,Steel,ES0132105018,0.63
ACS.MC,ACS,Madrid,Construction,ES0167050915,1.88
AENA.MC,Aena,Madrid,Airports,ES0105046009,3.78
AMS.MC,Amadeus IT Group,Madrid,Tourism,ES0109067019,5.99
MTS.MC,ArcelorMittal,"Luxembourg City, Luxembourg",Steel,LU1598757687,1.14
SAB.MC,Banco Sabadell,Alicante,Financial Services,ES0113860A34,0.59
SAN.MC,Santander,"Santander, Cantabria",Financial Services,ES0113900J37,11.19
BKT.MC,Bankinter,Madrid,Financial Services,ES0113679I37,1.12


## Nikkei 225 Index

In [37]:
nikkei_url = 'https://indexes.nikkei.co.jp/en/nkave/index/component'
nikkei_response = requests.get(nikkei_url)
print(nikkei_response.status_code)

200


In [38]:
nikkei_soup = BeautifulSoup(nikkei_response.text, 'html.parser')
nikkei_table = nikkei_soup.find_all("div", class_ = ["row component-list","row component-list list-even"])

In [39]:
nikkei_df = pd.DataFrame(nikkei_table, dtype = object)
nikkei_df = nikkei_df.drop(labels = [0,2,4], axis = 1)
nikkei_df.columns = ['Symbol', 'Security']
for i in range(len(nikkei_df['Symbol'])):
    nikkei_df['Symbol'][i] = str(nikkei_df['Symbol'][i])
    nikkei_df['Security'][i] = str(nikkei_df['Security'][i])
nikkei_pattern = "\>(.*?)\<"
for i in range(len(nikkei_df['Symbol'])):
    nikkei_df['Symbol'][i] = re.search(nikkei_pattern, nikkei_df['Symbol'][i]).group(1) + '.T'
    nikkei_df['Security'][i] = re.search(nikkei_pattern, nikkei_df['Security'][i]).group(1)
nikkei_df['Security'] = nikkei_df['Security'].str.replace('amp;','',regex = True)
nikkei_df = nikkei_df.set_index('Symbol')
nikkei_df

,Security
Symbol,
4151.T,"KYOWA KIRIN CO., LTD."
4502.T,"TAKEDA PHARMACEUTICAL CO., LTD."
4503.T,ASTELLAS PHARMA INC.
4506.T,"SUMITOMO PHARMA CO., LTD."
4507.T,"SHIONOGI & CO., LTD."
...,...
9501.T,"TOKYO ELECTRIC POWER COMPANY HOLDINGS, I"
9502.T,"CHUBU ELECTRIC POWER CO., INC."
9503.T,"THE KANSAI ELECTRIC POWER CO., INC."


## KOSPI Composite

In [40]:
kospi_url = 'http://topforeignstocks.com/indices/the-components-of-the-korea-stock-exchange-kospi-index/'
kospi_response = requests.get(kospi_url)
print(kospi_response.status_code)

200


In [41]:
kospi_soup = BeautifulSoup(kospi_response.text, 'html5lib')
kospi_table = kospi_soup.find_all("table")

In [42]:
kospi_df = pd.read_html(str(kospi_table))
kospi_df = np.asarray(kospi_df)
kospi_df = kospi_df.reshape(np.shape(kospi_df)[1], np.shape(kospi_df)[2])
kospi_df = pd.DataFrame(kospi_df)
kospi_df.rename({2:'Symbol', 1:'Security'}, axis = 'columns', inplace = True)
kospi_df['Symbol'] = kospi_df['Symbol'].astype(str)
for i, symbol in enumerate(kospi_df['Symbol']):
    if len(symbol) != 6:
        kospi_df['Symbol'].iloc[i] = '0' * (6 - len(symbol)) + symbol
for i, symbol in enumerate(kospi_df['Symbol']):
    kospi_df['Symbol'][i] = symbol + '.KS'
kospi_df = kospi_df[['Symbol', 'Security']]
kospi_df = kospi_df.set_index('Symbol')
kospi_df

,Security
Symbol,
015260.KS,A & P
244920.KS,A Plus Asset
140910.KS,A REIT
078520.KS,ABLE C&C
018250.KS,AEKYUNG INDUSTRIAL
...,...
000100.KS,YUHAN
003460.KS,YUHWA SECU
003520.KS,YUNGJIN PHARM


## TSEC Weighted

In [43]:
tsec_url = 'https://topforeignstocks.com/indices/the-components-of-the-taiwan-stock-exchange-weighted-index/'
tsec_response = requests.get(tsec_url)
print(tsec_response.status_code)

200


In [44]:
tsec_soup = BeautifulSoup(tsec_response.text, 'html5lib')
tsec_table = tsec_soup.find_all("table")

In [45]:
tsec_df = pd.read_html(str(tsec_table))
tsec_df = np.asarray(tsec_df)
tsec_df = tsec_df.reshape(np.shape(tsec_df)[1], np.shape(tsec_df)[2])
tsec_df = pd.DataFrame(tsec_df)
tsec_df.rename({2:'Symbol', 1:'Security'}, axis = 'columns', inplace = True)
tsec_df['Symbol'] = tsec_df['Symbol'].astype(str)
for i, symbol in enumerate(tsec_df['Symbol']):
    if len(symbol) != 4:
        tsec_df['Symbol'].iloc[i] = '0' * (4 - len(symbol)) + symbol
for i, symbol in enumerate(tsec_df['Symbol']):
    tsec_df['Symbol'][i] = symbol + '.TWO'
tsec_df = tsec_df[['Symbol', 'Security']]
tsec_df = tsec_df.set_index('Symbol')
tsec_df

,Security
Symbol,
8043.TWO,3H
3259.TWO,3S
3438.TWO,AAT
3236.TWO,ABC
8071.TWO,ABICO
...,...
9962.TWO,YYS
8176.TWO,ZCOM
6679.TWO,ZILLTEK


## Hang Seng

In [46]:
hs_url = 'http://www.etnet.com.hk/www/eng/stocks/indexes_detail.php?subtype=HSI'
hs_response = requests.get(hs_url)
print(hs_response.status_code)

200


In [47]:
hs_soup = BeautifulSoup(hs_response.text, 'html5lib')
hs_rows = hs_soup.find("table", class_ = ["figureTable"])

In [48]:
hs_df = pd.read_html(str(hs_rows))
hs_df = np.asarray(hs_df)
hs_df = hs_df.reshape(np.shape(hs_df)[1], np.shape(hs_df)[2])
hs_df = pd.DataFrame(hs_df)
hs_df.columns = hs_df.iloc[0]
hs_df.drop(hs_df.index[0], inplace = True)
hs_df.rename({'Code':'Symbol', 'Name':'Security'}, axis = 'columns', inplace = True)
hs_df = hs_df[['Symbol', 'Security']]
for i, ticker in enumerate(hs_df.Symbol):
    hs_df.Symbol[i+1] = ticker[1:5] + '.HK'
hs_df.set_index('Symbol', inplace = True)
hs_df

,Security
Symbol,
0001.HK,CKH HOLDINGS
0002.HK,CLP HOLDINGS
0003.HK,HK & CHINA GAS
0005.HK,HSBC HOLDINGS
0006.HK,POWER ASSETS
...,...
6862.HK,HAIDILAO
9618.HK,JD-SW
9633.HK,NONGFU SPRING


## CSI 300 Index

In [49]:
csi_url = 'https://en.wikipedia.org/wiki/CSI_300_Index'
csi_response = requests.get(csi_url)
print(csi_response.status_code)

200


In [50]:
csi_soup = BeautifulSoup(csi_response.text, 'html.parser')
csi_table = csi_soup.find_all('table',{'class':"wikitable"})

In [51]:
csi_df = pd.read_html(str(csi_table[1]))
csi_df = pd.DataFrame(csi_df[0])
csi_df['Index'] = csi_df['Index'].astype(str)
csi_df.rename({'Index':'Symbol', 'Company':'Security'}, axis = 'columns', inplace = True)
for i, symbol in enumerate(csi_df['Symbol']):
    if len(symbol) != 6:
        csi_df['Symbol'].iloc[i] = '0' * (6 - len(symbol)) + symbol
for i, symbol in enumerate(csi_df['Symbol']):
    if csi_df['Stock exchange'].iloc[i] == 'Shenzhen':
        csi_df['Symbol'].iloc[i] = symbol + '.SZ'
    else:
        csi_df['Symbol'].iloc[i] = symbol + '.SS'
csi_df = csi_df.set_index('Symbol')
csi_df

,Security,Stock exchange,Weighting (%),Segment
Symbol,,,,
601318.SS,Ping An Insurance,Shanghai,407,Financials
600016.SS,China Minsheng Bank,Shanghai,254,Financials
601166.SS,Industrial Bank,Shanghai,242,Financials
600036.SS,China Merchants Bank,Shanghai,206,Financials
601328.SS,Bank of Communications,Shanghai,181,Financials
...,...,...,...,...
600022.SS,Shandong Iron and Steel Company,Shanghai,5,Materials
600188.SS,Yanzhou Coal Mining,Shanghai,5,Energy
603885.SS,Juneyao Airlines,Shanghai,5,Industrials


## Straits Times Index

In [52]:
st_url = 'https://en.wikipedia.org/wiki/Straits_Times_Index'
st_response = requests.get(st_url)
print(st_response.status_code)

200


In [53]:
st_soup = BeautifulSoup(st_response.text, 'html.parser')
st_table = st_soup.find_all('table',{'class':"wikitable"})
# st_table[2]

In [54]:
st_df = pd.read_html(str(st_table))
st_df = pd.DataFrame(st_df[2])
st_df.rename({'Stock Symbol':'Symbol', 'Company':'Security'}, axis = 'columns', inplace = True)
for i, ticker in enumerate(st_df.Symbol):
    st_df['Symbol'][i] = re.sub('SGX: ', '', ticker) + '.SI'
st_df = st_df.set_index('Symbol')
st_df

,Security
Symbol,
A17U.SI,Ascendas Real Estate Investment Trust
C38U.SI,CapitaLand Integrated Commercial Trust
9CI.SI,CapitaLand Investment
C09.SI,City Developments Limited
C52.SI,ComfortDelGro Corporation
D01.SI,Dairy Farm International Holdings
D05.SI,DBS Group Holdings
BUOU.SI,Frasers Logistics & Commercial Trust
G13.SI,Genting Singapore PLC


## NIFTY 50

In [55]:
nifty_url = 'https://en.wikipedia.org/wiki/NIFTY_50'
nifty_response = requests.get(nifty_url)
print(nifty_response.status_code)

200


In [56]:
nifty_soup = BeautifulSoup(nifty_response.text, 'html.parser')
nifty_table = nifty_soup.find_all('table',{'class':"wikitable"})

In [57]:
nifty_df = pd.read_html(str(nifty_table))
nifty_df = pd.DataFrame(nifty_df[0])
nifty_df.rename({'Stock Symbol':'Symbol', 'Company Name':'Security'}, axis = 'columns', inplace = True)
for i, ticker in enumerate(nifty_df.Symbol):
    nifty_df['Symbol'][i] = re.sub('SGX: ', '', ticker) + '.NS'
nifty_df = nifty_df.set_index('Symbol')
nifty_df

,Security,Sector
Symbol,,
ADANIPORTS.NS,Adani Ports,Infrastructure
APOLLOHOSP.NS,Apollo Hospitals,Healthcare
ASIANPAINT.NS,Asian Paints,Consumer Goods
AXISBANK.NS,Axis Bank,Banking
BAJAJ-AUTO.NS,Bajaj Auto,Automobile
BAJFINANCE.NS,Bajaj Finance,Financial Services
BAJAJFINSV.NS,Bajaj Finserv,Financial Services
BHARTIARTL.NS,Bharti Airtel,Telecommunication
BPCL.NS,Bharat Petroleum,Energy - Oil & Gas


## BSE SENSEX

In [58]:
bse_url = 'https://en.wikipedia.org/wiki/BSE_SENSEX'
bse_response = requests.get(bse_url)
print(bse_response.status_code)

200


In [59]:
bse_soup = BeautifulSoup(bse_response.text, 'html.parser')
bse_table = bse_soup.find_all('table',{'class':"wikitable"})

In [60]:
bse_df = pd.read_html(str(bse_table))
bse_df = pd.DataFrame(bse_df[0])
bse_df.rename({'Stock Symbol':'Symbol', 'Companies':'Security'}, axis = 'columns', inplace = True)
bse_df = bse_df.set_index('Symbol')
bse_df.drop(['#', 'Exchange ticker'], axis = 1, inplace = True)
bse_df

,Security,Sector,Date Added
Symbol,,,
ASIANPAINT.BO,Asian Paints,Paints,21 December 2015[4]
AXISBANK.BO,Axis Bank,Banking - Private,NaN
BAJAJ-AUTO.BO,Bajaj Auto,Automobile,NaN
BAJFINANCE.BO,Bajaj Finance,Finance (NBFC),24 December 2018[5]
BAJAJFINSV.BO,Bajaj Finserv,Finance (Investment),NaN
BHARTIARTL.BO,Bharti Airtel,Telecommunications,NaN
DRREDDY.BO,Dr Reddy's Laboratories,Pharmaceuticals,NaN
HCLTECH.BO,HCL Technologies,IT Services & Consulting,NaN
HDFC.BO,HDFC,Finance (Housing),NaN


## MSCI Brazil

In [61]:
brz_df = pd.read_csv('https://www.ishares.com/us/products/239612/ishares-msci-brazil-capped-etf/1467271812596.ajax?fileType=csv&fileName=EWZ_holdings&dataType=fund', skiprows = 9)
brz_df.dropna(inplace = True)
brz_df = brz_df[brz_df['Sector'] != 'Cash and/or Derivatives']
brz_df.rename({'Ticker':'Symbol', 'Name':'Security'}, axis = 'columns', inplace = True)
brz_df['Symbol'] = brz_df['Symbol'].str.replace('*','',regex = True)
for i, ticker in enumerate(brz_df.Symbol):
    brz_df.Symbol[i] = ticker + '.SA'
brz_df = brz_df.set_index('Symbol')
brz_df = brz_df[['Security']]
brz_df

,Security
Symbol,
VALE3.SA,CIA VALE DO RIO DOCE SH
PETR4.SA,PETROLEO BRASILEIRO PREF SA
PETR3.SA,PETROLEO BRASILEIRO SA PETROBRAS
ITUB4.SA,ITAU UNIBANCO HOLDING PREF SA
ABEV3.SA,AMBEV SA
BBDC4.SA,BANCO BRADESCO PREF SA
B3SA3.SA,B3 BRASIL BOLSA BALCAO SA
WEGE3.SA,WEG SA
RENT3.SA,LOCALIZA RENT A CAR SA


## TA 125

In [62]:
ta_url = 'https://en.wikipedia.org/wiki/TA-125_Index'
ta_response = requests.get(ta_url)
print(ta_response.status_code)

200


In [63]:
ta_soup = BeautifulSoup(ta_response.text, 'html.parser')
ta_table = ta_soup.find_all('table',{'class':"wikitable"})

In [64]:
ta_df = pd.read_html(str(ta_table))
ta_df = pd.DataFrame(ta_df[1])
ta_df.rename({'Name':'Security'}, axis = 'columns', inplace = True)
for i, ticker in enumerate(ta_df.Symbol):
    ta_df['Symbol'][i] = ticker + '.TA'
ta_df = ta_df.set_index('Symbol')
ta_df = ta_df[['Security']]
ta_df

,Security
Symbol,
NICE.TA,NICE Ltd.
POLI.TA,Bank Hapoalim Ltd.
LUMI.TA,Bank Leumi Ltd.
PRGO.TA,Perrigo Ltd.
TEVA.TA,Teva Pharmaceutical Industries Ltd.
...,...
LVPR.TA,Levinstein Properties Ltd.
HAP.TA,Hadera Paper Ltd.
FBRT.TA,FMS Enterprises Migun Ltd.


# Analysis

In [65]:
class OvI:
    start = dt.datetime(2010, 1, 1)
    end = dt.datetime(2022, 7, 23)
    
    def __init__(self, ticker_table):
        self.tickers = ticker_table
        self.data = self.download(self.tickers)
        self.together = self.together()
        self.top_quint = self.top_quint()
        
    def download(self, ticker_table):
        data = yf.download(list(ticker_table.index), start = OvI.start, end = OvI.end)[['Open','Close']]
        data.columns = data.columns.swaplevel()
        data = data.reindex(sorted(data.columns), axis = 1)
        data.dropna(axis = 1, how = 'all', inplace = True)
        return data
    
    def overnight(self, opens, closes):
        ov_closes = closes.shift(1)
        df = pd.DataFrame(index = ['Overnight'])
        for i, ticker in enumerate(ov_closes.columns):
            pct_change = (opens[[ticker]] - ov_closes[[ticker]])/ov_closes[[ticker]]
            last = pct_change.apply(lambda x: (x + 1).cumprod()).iloc[-1]
            df.insert(i, ticker, last[0])
            df = df.copy()
        return df
    
    def intraday(self, opens, closes):
        df = pd.DataFrame(index = ['Intraday'])
        for i, ticker in enumerate(closes.columns):
            pct_change = (closes[[ticker]] - opens[[ticker]])/opens[[ticker]]
            last = pct_change.apply(lambda x: (x + 1).cumprod()).iloc[-1]
            df.insert(i, ticker, last[0])
            df = df.copy()
        return df

    def total(self, opens, closes):
        df = pd.DataFrame(index = ['Total'])
        for i, ticker in enumerate(closes.columns):
            first_open = opens[[ticker]].loc[opens[[ticker]].first_valid_index()][0]
            pct_change = (closes[[ticker]].iloc[-1] - first_open)/first_open
            df.insert(i, ticker, pct_change[0])
            df = df.copy()
        return df

    def together(self):
        opens = self.data.xs('Open', axis = 1, level = 1)
        closes = self.data.xs('Close', axis = 1, level = 1)
        df_o = self.overnight(opens, closes)
        df_i = self.intraday(opens, closes)
        df_t = self.total(opens, closes)
        df_a = pd.concat([df_o,df_i,df_t])
        df_a = df_a.transpose()
        df_a.insert(0, 'Security', self.tickers.sort_index()['Security'])
        df_a['O > I'] = np.where(df_a['Overnight'] > df_a['Intraday'], True, False)
        df_a['O > T'] = np.where(df_a['Overnight'] > df_a['Total'], True, False)
        df_a['O > I and T'] = (df_a.iloc[:, 2:4].values < df_a[['Overnight']].values).all(axis=1)
        return df_a
    
    def top_quint(self):
        df_to = self.together
        df_o_greater = df_to.loc[df_to['O > I'] == True]
        quintile_length = int(np.floor(len(df_o_greater) / 5))
        top_quint = df_o_greater.sort_values(by = 'Overnight', axis = 0, ascending = False)[:quintile_length][['Security']]
        return top_quint

## S&P 500

In [66]:
sp = OvI(sp_df)
sp.data

[*********************100%***********************]  503 of 503 completed


A                AAL                AAP              \
                 Close        Open  Close   Open       Close        Open   
Date                                                                       
2010-01-04   22.389128   22.453505   4.77   4.84   40.380001   40.700001   
2010-01-05   22.145924   22.324751   5.31   4.79   40.139999   40.299999   
2010-01-06   22.067240   22.067240   5.09   5.19   40.490002   40.049999   
2010-01-07   22.038628   22.017166   5.24   5.06   40.480000   39.549999   
2010-01-08   22.031473   21.917025   5.14   5.27   40.639999   40.250000   
...                ...         ...    ...    ...         ...         ...   
2022-07-18  115.110001  118.949997  14.54  14.83  185.270004  190.000000   
2022-07-19  119.059998  116.989998  15.16  14.90  190.649994  188.300003   
2022-07-20  119.860001  119.930000  15.21  15.14  192.660004  191.570007   
2022-07-21  127.190002  122.120003  14.08  14.67  189.830002  191.580002   
2022-07-22  125.040001  127.410004  13.68  14.11  191.229996  191.830002   

                  AAPL                    ABBV              ...         YUM  \
                 Close        Open       Close        Open  ...       Close   
Date                                                        ...               
2010-01-04    7.643214    7.622500         NaN         NaN  ...   25.226456   
2010-01-05    7.656429    7.664286         NaN         NaN  ...   25.140186   
2010-01-06    7.534643    7.656429         NaN         NaN  ...   24.960461   
2010-01-07    7.520714    7.562500         NaN         NaN  ...   24.953272   
2010-01-08    7.570714    7.510714         NaN         NaN  ...   24.960461   
...                ...         ...         ...         ...  ...         ...   
2022-07-18  147.070007  150.740005  149.570007  153.800003  ...  118.480003   
2022-07-19  151.000000  147.919998  149.740005  150.000000  ...  119.910004   
2022-07-20  153.039993  151.119995  147.690002  149.529999  ...  120.809998   
2022-07-21  155.350006  154.500000  147.750000  147.240005  ...  120.250000   
2022-07-22  154.089996  155.389999  148.470001  148.470001  ...  119.449997   

                               ZBH                    ZBRA              \
                  Open       Close        Open       Close        Open   
Date                                                                     
2010-01-04   25.248022   58.271847   57.582523   28.670000   28.469999   
2010-01-05   25.248022   60.116505   59.533981   28.620001   28.549999   
2010-01-06   25.125809   60.097088   60.252426   28.400000   27.410000   
2010-01-07   24.852625   61.475727   61.019417   27.690001   28.230000   
2010-01-08   24.946081   60.184467   60.009708   27.600000   27.580000   
...                ...         ...         ...         ...         ...   
2022-07-18  120.449997  103.480003  103.769997  295.839996  299.579987   
2022-07-19  119.160004  107.239998  105.040001  311.910004  301.089996   
2022-07-20  120.129997  108.000000  107.290001  325.420013  312.010010   
2022-07-21  120.430000  108.449997  108.290001  332.149994  324.700012   
2022-07-22  120.959999  107.000000  108.320000  326.679993  330.950012   

                 ZION                    ZTS              
                Close       Open       Close        Open  
Date                                                      
2010-01-04  13.330000  12.940000         NaN         NaN  
2010-01-05  13.800000  13.280000         NaN         NaN  
2010-01-06  15.000000  13.670000         NaN         NaN  
2010-01-07  16.680000  14.970000         NaN         NaN  
2010-01-08  16.410000  16.190001         NaN         NaN  
...               ...        ...         ...         ...  
2022-07-18  50.869999  51.130001  171.759995  174.839996  
2022-07-19  52.599998  51.459999  177.419998  173.979996  
2022-07-20  53.270000  52.169998  176.250000  177.539993  
2022-07-21  53.049999  53.029999  180.600006  175.770004  
2022-07-22  52.759998  53.040001  178.179993 

In [67]:
sp.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
A,Agilent Technologies,2.568851,2.167834,4.568841,True,False,False
AAL,American Airlines Group,151.777340,0.018622,1.826446,True,True,True
AAP,Advance Auto Parts,3.066848,1.532037,3.698526,True,False,False
AAPL,Apple Inc.,10.765382,1.877792,19.215152,True,False,False
ABBV,AbbVie,1.364462,3.116041,3.251718,False,False,False
...,...,...,...,...,...,...,...
YUM,Yum! Brands,1.422519,3.325836,3.731064,False,False,False
ZBH,Zimmer Biomet,0.879136,2.113669,0.858203,False,True,False
ZBRA,Zebra Technologies,3.371129,3.403766,10.474535,False,False,False
ZION,Zions Bancorporation,2.894665,1.408550,3.077280,True,False,False


In [68]:
sp.top_quint

,Security
ENPH,Enphase
TSLA,"Tesla, Inc."
AAL,American Airlines Group
MU,Micron Technology
MGM,MGM Resorts
AMD,AMD
SWKS,Skyworks Solutions
NVDA,Nvidia
F,Ford Motor Company
DAL,Delta Air Lines


In [69]:
time.sleep(180)

## NASDAQ 100

In [70]:
daq = OvI(daq_df)
daq.data

[*********************100%***********************]  102 of 102 completed


AAPL                    ABNB                    ADBE  \
                 Close        Open       Close        Open       Close   
Date                                                                     
2010-01-04    7.643214    7.622500         NaN         NaN   37.090000   
2010-01-05    7.656429    7.664286         NaN         NaN   37.700001   
2010-01-06    7.534643    7.656429         NaN         NaN   37.619999   
2010-01-07    7.520714    7.562500         NaN         NaN   36.889999   
2010-01-08    7.570714    7.510714         NaN         NaN   36.689999   
...                ...         ...         ...         ...         ...   
2022-07-18  147.070007  150.740005   97.669998   96.779999  375.230011   
2022-07-19  151.000000  147.919998  102.199997   99.699997  387.829987   
2022-07-20  153.039993  151.119995  107.730003  103.110001  401.489990   
2022-07-21  155.350006  154.500000  108.139999  106.980003  408.910004   
2022-07-22  154.089996  155.389999  103.970001  108.309998  401.899994   

                               ADI                     ADP              ...  \
                  Open       Close        Open       Close        Open  ...   
Date                                                                    ...   
2010-01-04   36.650002   31.670000   31.790001   37.603161   38.226513  ...   
2010-01-05   37.040001   31.620001   31.670000   37.401230   37.445126  ...   
2010-01-06   37.330002   31.559999   31.709999   37.313435   37.576820  ...   
2010-01-07   37.410000   31.309999   31.459999   37.295876   36.988586  ...   
2010-01-08   36.750000   31.490000   31.200001   37.243195   37.243195  ...   
...                ...         ...         ...         ...         ...  ...   
2022-07-18  383.829987  155.250000  158.839996  211.309998  214.279999  ...   
2022-07-19  382.890015  161.449997  156.929993  218.949997  214.250000  ...   
2022-07-20  392.190002  163.119995  159.649994  216.050003  218.460007  ...   
2022-07-21  404.119995  165.190002  164.380005  218.089996  215.740005  ...   
2022-07-22  410.029999  163.270004  165.660004  220.339996  219.110001  ...   

                  WBA                   WDAY                    XEL  \
                Close       Open       Close        Open      Close   
Date                                                                  
2010-01-04  37.299999  37.009998         NaN         NaN  21.080000   
2010-01-05  37.000000  37.099998         NaN         NaN  20.830000   
2010-01-06  36.720001  36.160000         NaN         NaN  20.870001   
2010-01-07  36.939999  36.689999         NaN         NaN  20.780001   
2010-01-08  36.990002  36.919998         NaN         NaN  20.790001   
...               ...        ...         ...         ...        ...   
2022-07-18  38.320000  38.330002  138.740005  142.740005  68.550003   
2022-07-19  38.599998  38.730000  141.839996  141.419998  68.860001   
2022-07-20  38.919998  38.580002  147.300003  143.179993  67.599998   
2022-07-21  38.459999  37.560001  149.779999  146.750000  67.699997   
2022-07-22  38.660000  38.770000  148.000000  149.949997  68.669998   

                               ZM                      ZS              
                 Open       Close        Open       Close        Open  
Date                                                                   
2010-01-04  21.379999         NaN         NaN         NaN         NaN  
2010-01-05  20.950001         NaN         NaN         NaN         NaN  
2010-01-06  20.870001         NaN         NaN         NaN         NaN  
2010-01-07  20.799999         NaN         NaN         NaN         NaN  
2010-01-08  20.760000         NaN         NaN         NaN         NaN  
...               ...         ...         ...         ...         ...  
2022-07-18  69.690002   98.919998  101.980003  150.820007  153.669998  
2022-07-19  69.199997  102.080002  100.400002  155.839996  153.479996  
2022-07-20  68.860001  107.089996  103.489998  162.949997  156.669998  
2022-07-21  67.

In [71]:
daq.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
AAPL,Apple Inc.,10.765382,1.877792,19.215152,True,False,False
ABNB,Airbnb,0.897951,0.793053,-0.287877,True,True,True
ADBE,Adobe Inc.,2.371977,4.623103,9.965893,False,False,False
ADI,Analog Devices,2.371255,2.165896,4.135892,True,False,False
ADP,ADP,0.898841,6.412770,4.764062,False,False,False
...,...,...,...,...,...,...,...
WBA,Walgreens Boots Alliance,0.734850,1.421491,0.044583,False,True,False
WDAY,"Workday, Inc.",1.023624,3.009040,2.080125,False,False,False
XEL,Xcel Energy,0.785791,4.087448,2.211880,False,False,False
ZM,Zoom Video Communications,1.745327,0.939211,0.639231,True,True,True


In [72]:
daq.top_quint

,Security
TSLA,"Tesla, Inc."
MU,Micron Technology
AMD,AMD
SWKS,Skyworks Solutions
NVDA,Nvidia
NXPI,NXP
BIDU,Baidu
MELI,MercadoLibre
AVGO,Broadcom Inc.
AMZN,Amazon


In [73]:
time.sleep(180)

## TSX 60

In [74]:
tsx = OvI(tsx_df)
tsx.data

[*********************100%***********************]  60 of 60 completed


ABX.TO                AEM.TO                AQN.TO             \
                Close       Open      Close       Open      Close       Open   
Date                                                                           
2010-01-04  42.080002  42.349998  58.650002  58.490002   4.130000   4.100000   
2010-01-05  42.480000  42.250000  58.869999  59.000000   4.160000   4.160000   
2010-01-06  43.180000  42.860001  60.160000  60.000000   4.070000   4.200000   
2010-01-07  42.639999  42.900002  59.709999  60.049999   4.180000   4.080000   
2010-01-08  42.740002  43.049999  61.369999  60.029999   4.260000   4.180000   
...               ...        ...        ...        ...        ...        ...   
2022-07-18  20.549999  20.629999  55.430000  56.330002  17.299999  17.400000   
2022-07-19  20.660000  20.459999  55.299999  55.590000  17.299999  17.370001   
2022-07-20  19.889999  20.590000  52.860001  55.419998  17.170000  17.320000   
2022-07-21  19.969999  20.010000  53.250000  52.860001  17.139999  17.190001   
2022-07-22  19.799999  20.120001  52.000000  53.939999  17.280001  17.200001   

               ATD.TO              BAM-A.TO             ...     TRP.TO  \
                Close       Open      Close       Open  ...      Close   
Date                                                    ...              
2010-01-04   3.401667   3.336667  10.003227  10.175325  ...  36.029999   
2010-01-05   3.401667   3.401667  10.140906  10.110788  ...  35.959999   
2010-01-06   3.335000   3.366667   9.955900  10.136603  ...  35.619999   
2010-01-07   3.390000   3.346667  10.024739   9.990319  ...  35.320000   
2010-01-08   3.348333   3.335000  10.050554  10.020437  ...  35.200001   
...               ...        ...        ...        ...  ...        ...   
2022-07-18  51.439999  53.000000  58.320000  58.820000  ...  67.669998   
2022-07-19  52.270000  51.840000  60.290001  59.009998  ...  68.169998   
2022-07-20  52.610001  52.139999  61.250000  60.130001  ...  67.940002   
2022-07-21  52.650002  52.470001  61.840000  61.250000  ...  67.970001   
2022-07-22  53.709999  52.680000  61.980000  61.990002  ...  68.629997   

                           WCN.TO             WEED.TO             WN.TO  \
                 Open       Close        Open   Close  Open       Close   
Date                                                                      
2010-01-04  36.270000   23.662167   23.496019     NaN   NaN   67.150002   
2010-01-05  36.150002   23.399099   23.745241     NaN   NaN   68.010002   
2010-01-06  36.049999   23.537556   23.399099     NaN   NaN   68.010002   
2010-01-07  35.639999   23.869850   23.703705     NaN   NaN   67.930000   
2010-01-08  35.349998   23.897543   23.869850     NaN   NaN   71.309998   
...               ...         ...         ...     ...   ...         ...   
2022-07-18  66.900002  159.369995  162.029999    3.36  2.98  153.419998   
2022-07-19  67.690002  161.289993  160.100006    3.35  3.40  154.070007   
2022-07-20  67.949997  161.289993  161.449997    3.93  3.35  154.080002   
2022-07-21  67.699997  164.270004  161.750000    3.57  4.03  154.210007   
2022-07-22  68.080002  163.149994  164.710007    3.31  3.49  155.149994   

                           WPM.TO             
                  Open      Close       Open  
Date                                          
2010-01-04   67.040001  16.430000  16.379999  
2010-01-05   67.489998  16.760000  16.549999  
2010-01-06   67.919998  17.299999  17.070000  
2010-01-07   68.000000  17.260000  17.299999  
2010-01-08   67.930000  17.660000  17.410000  
...                ...        ...        ...  
2022-07-18  153.839996  43.939999  44.529999  
2022-07-19  153.419998  43.759998  43.990002  
2022-07-20  153.699997  41.610001  43.799999  
2022-07-21  153.380005  42.110001  41.889999  
2022-07-22  154.300003  41.970001  42.450001  

[3151 rows x 120 columns]

In [75]:
tsx.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
ABX.TO,Barrick Gold Corporation,17.021336,0.027467,-0.532468,True,True,True
AEM.TO,Agnico Eagle Mines Limited,3.256648,0.272993,-0.110959,True,True,True
AQN.TO,Algonquin Power & Utilities Corp.,2.332908,1.806601,3.214634,True,False,False
ATD.TO,Alimentation Couche-Tard Inc.,23.391987,0.688137,15.096901,True,True,True
BAM-A.TO,Brookfield Asset Management Inc.,3.028207,2.011489,5.091206,True,False,False
BCE.TO,BCE Inc.,1.517699,1.429404,1.169405,True,True,True
BHC.TO,Bausch Health Companies Inc.,3.014780,0.255657,-0.229252,True,True,True
BIP-UN.TO,Brookfield Infrastructure Partners L.P.,240.492646,0.043573,9.479044,True,True,True
BMO.TO,Bank of Montreal,2.504544,0.893305,1.237321,True,True,True
BNS.TO,Bank of Nova Scotia,2.434610,0.623589,0.518196,True,True,True


In [76]:
tsx.top_quint

,Security
WEED.TO,Canopy Growth Corporation
BIP-UN.TO,Brookfield Infrastructure Partners L.P.
SHOP.TO,Shopify Inc.
ATD.TO,Alimentation Couche-Tard Inc.
ABX.TO,Barrick Gold Corporation
FNV.TO,Franco-Nevada Corporation


In [77]:
time.sleep(180)

## MSCI Mexico

In [78]:
mex = OvI(mex_df)
mex.data

[*********************100%***********************]  43 of 43 completed


AC.MX                AGUA.MX            ALFAA.MX          \
                 Close        Open      Close       Open    Close    Open   
Date                                                                        
2010-01-04   43.500000   43.450001        NaN        NaN    8.778   8.342   
2010-01-05   43.500000   43.500000        NaN        NaN    8.600   8.750   
2010-01-06   43.299999   43.500000        NaN        NaN    8.650   8.609   
2010-01-07   43.590000   43.400002        NaN        NaN    8.629   8.677   
2010-01-08   42.799999   43.480000        NaN        NaN    8.556   8.627   
...                ...         ...        ...        ...      ...     ...   
2022-07-18  137.639999  137.039993  24.629999  24.150000   14.940  14.750   
2022-07-19  136.630005  138.289993  24.840000  24.750000   14.950  14.990   
2022-07-20  129.990005  136.399994  24.830000  24.879999   14.590  14.930   
2022-07-21  131.160004  130.699997  24.559999  25.299999   14.660  15.000   
2022-07-22  138.070007  130.910004  24.750000  24.830000   14.360  14.690   

             ALSEA.MX               AMXL.MX             ... TLEVISACPO.MX  \
                Close       Open      Close       Open  ...         Close   
Date                                                    ...                 
2010-01-04   9.935226   9.935226  15.725000  15.490000  ...     54.459999   
2010-01-05   9.944966   9.944966  15.610000  15.650000  ...     53.720001   
2010-01-06   9.935226   9.935226  15.570000  15.645000  ...     53.209999   
2010-01-07   9.876783   9.876783  15.780000  15.500000  ...     54.049999   
2010-01-08   9.876783   9.876783  15.675000  15.635000  ...     52.810001   
...               ...        ...        ...        ...  ...           ...   
2022-07-18  38.500000  37.689999  19.280001  19.309999  ...     32.730000   
2022-07-19  38.750000  38.700001  19.549999  19.410000  ...     32.930000   
2022-07-20  38.720001  38.450001  18.990000  19.450001  ...     33.060001   
2022-07-21  38.029999  38.869999  19.160000  19.080000  ...     32.910000   
2022-07-22  37.869999  38.000000  19.200001  19.170000  ...     32.540001   

                      TRAXIONA.MX              VESTA.MX             VOLARA.MX  \
                 Open       Close       Open      Close       Open      Close   
Date                                                                            
2010-01-04  54.459999         NaN        NaN        NaN        NaN        NaN   
2010-01-05  53.720001         NaN        NaN        NaN        NaN        NaN   
2010-01-06  53.209999         NaN        NaN        NaN        NaN        NaN   
2010-01-07  54.049999         NaN        NaN        NaN        NaN        NaN   
2010-01-08  52.810001         NaN        NaN        NaN        NaN        NaN   
...               ...         ...        ...        ...        ...        ...   
2022-07-18  32.799999   26.430000  25.990000  40.290001  40.000000  19.010000   
2022-07-19  32.980000   26.490000  26.430000  39.990002  40.580002  19.709999   
2022-07-20  33.250000   26.469999  26.340000  39.470001  39.980000  21.309999   
2022-07-21  33.119999   27.170000  26.790001  40.040001  39.500000  21.469999   
2022-07-22  32.840000   26.150000  27.750000  40.240002  39.900002  18.670000   

                       WALMEX.MX             
                 Open      Close       Open  
Date                                         
2010-01-04        NaN  29.875000  29.950001  
2010-01-05        NaN  30.395000  29.780001  
2010-01-06        NaN  30.495001  30.350000  
2010-01-07        NaN  29.885000  30.500000  
2010-01-08        NaN  29.650000  29.879999  
...               ...        ...        ...  
2022-07-18  19.100000  71.690002  73.180000  
2022-07-19  19.240000  70.930000  71.900002  
2022-07-20  19.709999  71.290001  71.320000  
2022-07-21  21.230000  72.599998  71.639999  
2022-07-22  21.309999  73.279999  72.930000  

[3154 rows x 86 columns]

In [79]:
mex.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
AC.MX,ARCA CONTINENTAL,3.985398,0.785211,2.177676,True,True,True
AGUA.MX,GRUPO ROTOPLAS,14.880715,0.055441,-0.175000,True,True,True
ALFAA.MX,ALFA A,18.024367,0.095607,0.721410,True,True,True
ALSEA.MX,ALSEA DE CV,4.526000,0.848471,2.811690,True,True,True
AMXL.MX,AMERICA MOVIL L,0.920052,1.336000,0.239509,False,True,False
ASURB.MX,GRUPO AEROPORTUARIO DEL SURESTE B,5.491249,0.981491,4.375792,True,True,True
BBAJIOO.MX,BANCO DEL BAJIO INSTITUCION DE BAN,3.462256,0.425952,0.474754,True,True,True
BIMBOA.MX,GRUPO BIMBO A,1.224451,2.837800,2.440324,False,False,False
BOLSAA.MX,BOLSA MEXICANA DE VALORES,17.781021,0.131734,1.281008,True,True,True
CEMEXCPO.MX,CEMEX CPO,14.794159,0.047476,-0.278869,True,True,True


In [80]:
mex.top_quint

,Security
VOLARA.MX,CONTROLADORA VUELA COMPANIA DE AVI
ALFAA.MX,ALFA A
BOLSAA.MX,BOLSA MEXICANA DE VALORES
NEMAKA.MX,NEMAK
PE&OLES.MX,INDUST PENOLES
FUNO11.MX,FIBRA UNO ADMINISTRACION REIT SA
AGUA.MX,GRUPO ROTOPLAS


In [81]:
time.sleep(180)

## ASX 200

In [82]:
asx = OvI(asx_df)
asx.data

[*********************100%***********************]  200 of 200 completed


A2M.AX       ABC.AX          ABP.AX               AGL.AX  \
            Close  Open  Close  Open     Close      Open      Close   
Date                                                                  
2010-01-04    NaN   NaN   2.73  2.74  2.266161  2.241259  13.182467   
2010-01-05    NaN   NaN   2.72  2.74  2.315967  2.266161  13.219574   
2010-01-06    NaN   NaN   2.65  2.65  2.315967  2.315967  13.293789   
2010-01-07    NaN   NaN   2.65  2.64  2.340870  2.291064  13.173190   
2010-01-08    NaN   NaN   2.63  2.65  2.266161  2.266161  13.117528   
...           ...   ...    ...   ...       ...       ...        ...   
2022-07-18   4.40  4.53   2.38  2.33  2.760000  2.850000   8.150000   
2022-07-19   4.32  4.40   2.37  2.34  2.780000  2.790000   8.150000   
2022-07-20   4.46  4.38   2.44  2.40  2.810000  2.820000   8.370000   
2022-07-21   4.48  4.50   2.47  2.44  2.810000  2.800000   8.450000   
2022-07-22   4.56  4.48   2.49  2.49  2.810000  2.800000   8.410000   

                         AIA.AX            ...     WOW.AX            WPR.AX  \
                 Open     Close      Open  ...      Close       Open  Close   
Date                                       ...                                
2010-01-04  13.080421  1.759961  1.771030  ...  27.847349  27.867147    NaN   
2010-01-05  13.265959  1.820840  1.793168  ...  27.679056  28.124535    NaN   
2010-01-06  13.219574  1.793168  1.826374  ...  27.629559  27.560263    NaN   
2010-01-07  13.293789  1.826374  1.804236  ...  27.570162  27.718655    NaN   
2010-01-08  13.182467  1.826374  1.826374  ...  27.807751  27.758253    NaN   
...               ...       ...       ...  ...        ...        ...    ...   
2022-07-18   8.000000  6.660000  6.680000  ...  37.419998  37.599998   2.52   
2022-07-19   8.100000  6.610000  6.600000  ...  37.040001  37.349998   2.51   
2022-07-20   8.390000  6.750000  6.670000  ...  37.139999  37.000000   2.55   
2022-07-21   8.300000  6.650000  6.710000  ...  37.500000  37.160000   2.52   
2022-07-22   8.420000  6.560000  6.680000  ...  37.250000  37.299999   2.54   

                     WTC.AX                XRO.AX               ZIP.AX  \
            Open      Close       Open      Close       Open     Close   
Date                                                                     
2010-01-04   NaN        NaN        NaN        NaN        NaN  1.656250   
2010-01-05   NaN        NaN        NaN        NaN        NaN  1.623125   
2010-01-06   NaN        NaN        NaN        NaN        NaN  1.623125   
2010-01-07   NaN        NaN        NaN        NaN        NaN  1.590000   
2010-01-08   NaN        NaN        NaN        NaN        NaN  1.523750   
...          ...        ...        ...        ...        ...       ...   
2022-07-18  2.50  47.290001  45.099998  88.029999  86.599998  0.590000   
2022-07-19  2.52  44.970001  45.849998  82.730003  86.510002  0.590000   
2022-07-20  2.53  47.490002  46.080002  87.639999  86.540001  0.665000   
2022-07-21  2.54  49.180000  47.639999  90.500000  88.000000  0.775000   
2022-07-22  2.53  49.209999  49.500000  91.220001  90.750000  0.880000   

                      
                Open  
Date                  
2010-01-04  1.623125  
2010-01-05  1.623125  
2010-01-06  1.623125  
2010-01-07  1.590000  
2010-01-08  1.523750  
...              ...  
2022-07-18  0.585000  
2022-07-19  0.585000  
2022-07-20  0.610000  
2022-07-21  0.685000  
2022-07-22  0.775000  

[3176 rows x 400 columns]

In [83]:
asx.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
A2M.AX,The a2 Milk Company,34.568116,0.237682,7.216216,True,True,True
ABC.AX,Adbri,0.562874,1.737607,-0.091241,False,True,False
ABP.AX,Abacus Property Group,0.361460,3.605007,0.253760,False,True,False
AGL.AX,AGL Energy,0.646643,1.014025,-0.357054,False,True,False
AIA.AX,Auckland Airport,0.706170,5.259030,2.704059,False,False,False
...,...,...,...,...,...,...,...
WOW.AX,Woolworths,0.833593,1.655245,0.336699,False,True,False
WPR.AX,Waypoint REIT[15],2.196198,0.462618,0.016000,True,True,True
WTC.AX,Wisetech Global,1.021312,14.129942,13.431084,False,False,False
XRO.AX,Xero,20.834607,0.972954,19.271111,True,True,True


In [84]:
asx.top_quint

,Security
DEG.AX,De Grey Mining
BRN.AX,Brainchip
IMU.AX,Imugene
LTR.AX,Liontown Resources
PDN.AX,Paladin Energy
CXO.AX,Core Lithium
GOR.AX,Gold Road Res
LKE.AX,Lake Resources
LYC.AX,Lynas
RMS.AX,Ramelius Resources


In [85]:
time.sleep(180)

## OBX

In [86]:
obx = OvI(obx_df)
obx.data

[*********************100%***********************]  25 of 25 completed

3 Failed downloads:
- AKERBP.OL: No data found, symbol may be delisted
- SSO.OL: No data found, symbol may be delisted
- SCH.OL: No data found for this date range, symbol may be delisted


AKER.OL        BAKKA.OL          BWLPG.OL                BWO.OL  \
             Close   Open    Close   Open      Close       Open      Close   
Date                                                                         
2010-01-04   167.5  161.5      NaN    NaN        NaN        NaN  93.558220   
2010-01-05   167.0  168.0      NaN    NaN        NaN        NaN  95.034897   
2010-01-06   170.5  167.0      NaN    NaN        NaN        NaN  92.819878   
2010-01-07   168.5  170.0      NaN    NaN        NaN        NaN  93.452744   
2010-01-08   169.0  169.5      NaN    NaN        NaN        NaN  93.452744   
...            ...    ...      ...    ...        ...        ...        ...   
2022-07-18   726.0  718.0    624.5  628.0  75.000000  72.500000  25.760000   
2022-07-19   728.5  720.5    638.5  622.5  76.900002  74.500000  25.920000   
2022-07-20   737.5  734.5    656.5  642.5  75.150002  77.900002  25.920000   
2022-07-21   716.0  742.5    657.0  656.0  75.699997  76.099998  24.920000   
2022-07-22   728.0  723.5    667.5  655.5  76.599998  76.800003  25.680000   

                           DNB.OL              ...    SUBC.OL             \
                 Open       Close        Open  ...      Close       Open   
Date                                           ...                         
2010-01-04  89.655563   64.349998   63.200001  ...  94.599998  92.250000   
2010-01-05  94.929420   63.500000   64.349998  ...  94.400002  94.699997   
2010-01-06  96.511581   63.549999   63.700001  ...  93.400002  94.699997   
2010-01-07  90.815819   63.700001   63.349998  ...  95.250000  93.400002   
2010-01-08  93.874649   66.400002   64.199997  ...  95.050003  96.500000   
...               ...         ...         ...  ...        ...        ...   
2022-07-18  25.120001  185.000000  180.000000  ...  77.959999  76.279999   
2022-07-19  25.400000  187.850006  184.550003  ...  77.900002  76.019997   
2022-07-20  25.799999  187.800003  188.699997  ...  78.320000  79.820000   
2022-07-21  25.500000  186.399994  186.699997  ...  75.320000  79.300003   
2022-07-22  25.000000  186.250000  186.399994  ...  78.160004  76.739998   

                TEL.OL                  TGS.OL                  TOM.OL  \
                 Close        Open       Close        Open       Close   
Date                                                                     
2010-01-04   83.800003   81.199997  108.000000  106.000000   14.200000   
2010-01-05   83.400002   83.949997  107.500000  108.400002   14.250000   
2010-01-06   82.550003   83.849998  108.000000  109.000000   14.150000   
2010-01-07   82.300003   82.699997  110.400002  107.599998   14.000000   
2010-01-08   81.650002   82.349998  109.900002  111.000000   13.600000   
...                ...         ...         ...         ...         ...   
2022-07-18  132.699997  132.199997  138.300003  136.699997  200.000000   
2022-07-19  125.449997  132.250000  139.500000  138.000000  199.320007   
2022-07-20  123.349998  125.449997  137.800003  137.399994  201.399994   
2022-07-21  118.000000  122.000000  130.000000  140.699997  205.699997   
2022-07-22  116.400002  118.000000  134.500000  130.100006  205.300003   

                            YAR.OL              
                  Open       Close        Open  
Date                                            
2010-01-04   14.100000  272.200012  266.000000  
2010-01-05   14.200000  277.299988  274.000000  
2010-01-06   14.300000  271.700012  277.100006  
2010-01-07   14.100000  271.299988  272.100006  
2010-01-08   14.050000  273.399994  273.799988  
...                ...         ...         ...  
2022-07-18  194.000000  411.200012  414.299988  
2022-07-19  199.000000  416.399994  405.100006  
2022-07-20  201.000000  412.899994  419.899994  
2022-07-21  201.399994  410.799988  411.799988  
2022-07-22  206.100006  407.600006  410.000000  

[3152 rows x 44 columns]

In [87]:
obx.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
AKER.OL,Aker,6.415454,0.702638,3.507740,True,True,True
BAKKA.OL,Bakkafrost,5.260387,3.793296,18.954205,True,False,False
BWLPG.OL,BW LPG,58.067557,0.026122,0.516832,True,True,True
BWO.OL,BW Offshore,11857.036239,0.000024,-0.713570,True,True,True
DNB.OL,DNB,4.819272,0.606010,1.946994,True,True,True
DNO.OL,DNO,2221.019637,0.001122,1.491296,True,True,True
ENTRA.OL,Entra,3.130752,0.652531,1.042912,True,True,True
EQNR.OL,Equinor,1.777813,1.321078,1.348630,True,True,True
FRO.OL,Frontline,378.410206,0.000304,-0.885015,True,True,True
GJF.OL,Gjensidige Forsikring,2.664395,1.287537,2.430508,True,True,True


In [88]:
obx.top_quint

,Security
BWO.OL,BW Offshore
DNO.OL,DNO
NEL.OL,NEL
FRO.OL,Frontline


In [89]:
time.sleep(180)

## DAX

In [90]:
dax = OvI(dax_df)
dax.data

[*********************100%***********************]  40 of 40 completed


1COV.DE                 ADS.DE                  AIR.DE  \
                Close       Open       Close        Open       Close   
Date                                                                   
2010-01-04        NaN        NaN   38.505001   37.959999   14.000000   
2010-01-05        NaN        NaN   39.720001   39.000000   13.900000   
2010-01-06        NaN        NaN   39.400002   39.799999   13.995000   
2010-01-07        NaN        NaN   39.744999   39.365002   14.115000   
2010-01-08        NaN        NaN   39.599998   39.724998   14.345000   
...               ...        ...         ...         ...         ...   
2022-07-18  32.880001  32.599998  165.179993  161.600006  104.339996   
2022-07-19  34.790001  33.230000  175.919998  164.000000  105.879997   
2022-07-20  34.669998  35.009998  173.899994  176.559998  104.699997   
2022-07-21  33.959999  34.790001  175.080002  173.440002  104.739998   
2022-07-22  33.439999  33.709999  176.619995  174.820007  105.580002   

                            ALV.DE                 BAS.DE             ...  \
                  Open       Close        Open      Close       Open  ...   
Date                                                                  ...   
2010-01-04   13.895000   88.540001   87.800003  44.849998  43.459999  ...   
2010-01-05   13.840000   88.809998   88.500000  44.169998  44.764999  ...   
2010-01-06   13.975000   89.500000   89.150002  44.450001  44.220001  ...   
2010-01-07   13.965000   88.470001   88.919998  44.154999  44.169998  ...   
2010-01-08   14.370000   87.989998   89.180000  44.020000  44.250000  ...   
...                ...         ...         ...        ...        ...  ...   
2022-07-18  104.639999  173.679993  174.080002  42.720001  42.980000  ...   
2022-07-19  103.360001  175.720001  172.919998  44.685001  42.299999  ...   
2022-07-20  106.320000  175.139999  176.600006  44.665001  45.009998  ...   
2022-07-21  104.500000  174.380005  175.339996  43.669998  44.485001  ...   
2022-07-22  104.199997  174.119995  174.199997  43.139999  43.395000  ...   

               SRT3.DE                  SY1.DE                 VNA.DE  \
                 Close        Open       Close        Open      Close   
Date                                                                    
2010-01-04    4.175000    3.737500   15.515000   15.150000        NaN   
2010-01-05    4.037500    4.001250   15.520000   15.490000        NaN   
2010-01-06    4.075000    4.042500   16.020000   15.845000        NaN   
2010-01-07    3.960000    4.075000   15.910000   16.035000        NaN   
2010-01-08    3.975000    3.960000   16.059999   16.000000        NaN   
...                ...         ...         ...         ...        ...   
2022-07-18  370.899994  380.600006  105.699997  105.550003  28.650000   
2022-07-19  378.000000  367.399994  107.250000  105.400002  29.459999   
2022-07-20  383.200012  379.799988  106.849998  108.000000  29.570000   
2022-07-21  413.000000  387.000000  109.599998  106.349998  29.969999   
2022-07-22  418.500000  415.200012  111.050003  109.300003  31.920000   

                          VOW3.DE                 ZAL.DE             
                 Open       Close        Open      Close       Open  
Date                                                                 
2010-01-04        NaN   64.114349   64.760468        NaN        NaN  
2010-01-05        NaN   62.871826   64.114349        NaN        NaN  
2010-01-06        NaN   64.611359   63.209793        NaN        NaN  
2010-01-07        NaN   65.356880   64.581543        NaN        NaN  
2010-01-08        NaN   66.062630   65.505981        NaN        NaN  
...               ...         ...         ...        ...        ...  
2022-07-18  28.290001  135.960007  133.740005  27.020000  25.650000  
2022-07-19  28.469999  141.220001  134.699997  28.040001  26.440001  
2022-07-20  29.639999  139.619995  142.279999  28.510000  28.000000  
2022-07-21  29.500000  135.880005  139.919998  28.549999  28.48

In [91]:
dax.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
1COV.DE,Covestro,9.777293,0.129881,0.286154,True,True,True
ADS.DE,Adidas,5.363950,0.870185,3.652792,True,True,True
AIR.DE,Airbus,4.508580,1.703015,6.598417,True,False,False
ALV.DE,Allianz,4.100001,0.486178,0.983143,True,True,True
BAS.DE,BASF,2.414700,0.407896,-0.007363,True,True,True
BAYN.DE,Bayer,3.683786,0.289482,0.026647,True,True,True
BEI.DE,Beiersdorf,2.153663,1.021152,1.179280,True,True,True
BMW.DE,BMW,5.271734,0.460597,1.423004,True,True,True
BNR.DE,Brenntag,6.156979,0.624370,2.886497,True,True,True
CON.DE,Continental,6.141095,0.316606,0.960898,True,True,True


In [92]:
dax.top_quint

,Security
VOW3.DE,Volkswagen Group
IFX.DE,Infineon Technologies
HFG.DE,HelloFresh
HEI.DE,HeidelbergCement
PAH3.DE,Porsche
1COV.DE,Covestro


In [93]:
time.sleep(180)

## AEX

In [94]:
aex = OvI(aex_df)
aex.data

[*********************100%***********************]  25 of 25 completed


AD.AS                ADYEN.AS              AGN.AS         \
                Close       Open        Close         Open  Close   Open   
Date                                                                       
2010-01-04  10.854327  10.704691          NaN          NaN  4.809  4.579   
2010-01-05  10.763395  10.899218          NaN          NaN  4.798  4.840   
2010-01-06  10.824400  10.762244          NaN          NaN  4.781  4.810   
2010-01-07  10.587285  10.792171          NaN          NaN  4.878  4.760   
2010-01-08  10.696634  10.612608          NaN          NaN  5.029  4.935   
...               ...        ...          ...          ...    ...    ...   
2022-07-18  26.065001  26.000000  1550.000000  1517.000000  4.058  4.039   
2022-07-19  26.170000  25.940001  1577.800049  1525.400024  4.097  4.020   
2022-07-20  26.120001  26.330000  1596.199951  1588.800049  4.087  4.127   
2022-07-21  26.379999  26.400000  1662.400024  1600.000000  4.077  4.072   
2022-07-22  26.455000  26.299999  1622.000000  1653.199951  4.063  4.055   

              AKZA.AS                 ASM.AS              ...    TKWY.AS  \
                Close       Open       Close        Open  ...      Close   
Date                                                      ...              
2010-01-04  52.576866  52.424992   17.879999   17.770000  ...        NaN   
2010-01-05  52.177494  52.368744   18.025000   17.760000  ...        NaN   
2010-01-06  52.312492  52.171867   18.270000   18.000000  ...        NaN   
2010-01-07  51.463120  52.081856   17.840000   18.115000  ...        NaN   
2010-01-08  51.097492  51.671242   18.485001   18.155001  ...        NaN   
...               ...        ...         ...         ...  ...        ...   
2022-07-18  64.239998  64.180000  232.699997  237.100006  ...  14.600000   
2022-07-19  66.220001  63.799999  233.500000  227.449997  ...  14.972000   
2022-07-20  65.320000  64.220001  240.000000  231.500000  ...  16.330000   
2022-07-21  63.540001  64.180000  273.700012  246.800003  ...  16.323999   
2022-07-22  65.739998  64.599998  274.149994  273.299988  ...  18.572001   

                          UMG.AS                UNA.AS                 URW.AS  \
                 Open      Close       Open      Close       Open       Close   
Date                                                                            
2010-01-04        NaN        NaN        NaN        NaN        NaN  155.300003   
2010-01-05        NaN        NaN        NaN        NaN        NaN  155.100006   
2010-01-06        NaN        NaN        NaN        NaN        NaN  156.449997   
2010-01-07        NaN        NaN        NaN        NaN        NaN  154.800003   
2010-01-08        NaN        NaN        NaN        NaN        NaN  154.850006   
...               ...        ...        ...        ...        ...         ...   
2022-07-18  13.242000  21.135000  20.995001  45.660000  45.904999   49.369999   
2022-07-19  14.300000  21.475000  21.020000  46.275002  45.380001   50.430000   
2022-07-20  14.972000  21.485001  20.980000  45.715000  46.279999   49.849998   
2022-07-21  16.374001  21.584999  21.500000  45.599998  45.825001   48.959999   
2022-07-22  16.238001  21.420000  21.525000  45.924999  45.770000   52.020000   

                            WKL.AS              
                  Open       Close        Open  
Date                                            
2010-01-04  152.899994   15.600000   15.115000  
2010-01-05  154.850006   15.595000   15.580000  
2010-01-06  155.800003   15.555000   15.590000  
2010-01-07  156.149994   15.545000   15.530000  
2010-01-08  155.800003   15.495000   15.560000  
...                ...         ...         ...  
2022-07-18   49.244999   99.779999   99.660004  
2022-07-19   48.770000   99.419998   99.339996  
2022-07-20   50.590000   99.620003  102.000000  
2022-07-21   49.810001  102.250000   99.720001  
2022-07-22   48.994999  102.400002  102.099998  

[3215 rows x 50 columns]

In [95]:
aex.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
AD.AS,Ahold Delhaize,0.927477,2.664590,1.471346,False,False,False
ADYEN.AS,Adyen,1.428853,2.837941,3.055000,False,False,False
AGN.AS,Aegon,5.070108,0.175008,-0.112688,True,True,True
AKZA.AS,AkzoNobel,1.381507,0.907691,0.253982,True,True,True
ASM.AS,ASM International,27.551113,0.559966,14.427686,True,True,True
ASML.AS,ASML Holding,10.615644,1.585743,15.833682,True,False,False
BESI.AS,BE Semiconductors,15.431503,2.526059,37.980893,True,False,False
DSM.AS,DSM,1.987704,2.196675,3.366340,False,False,False
HEIA.AS,Heineken,2.469397,1.163594,1.873376,True,True,True
IMCD.AS,IMCD,7.020651,0.950119,5.670455,True,True,True


In [96]:
aex.top_quint

,Security
MT.AS,ArcelorMittal
ASM.AS,ASM International
INGA.AS,ING Group


In [97]:
time.sleep(180)

## CAC 40

In [98]:
cac = OvI(cac_df)
cac.data

[*********************100%***********************]  40 of 40 completed


ACA.PA                  AI.PA                  AIR.PA  \
                Close       Open       Close        Open       Close   
Date                                                                   
2010-01-04  12.349994  11.988883   47.727844   46.954033   13.990000   
2010-01-05  12.537772  12.325920   46.773289   47.445431   13.945000   
2010-01-06  12.759253  12.537772   46.485226   46.778938   14.070000   
2010-01-07  12.764068  12.672586   46.428745   46.146332   14.210000   
2010-01-08  12.971105  12.845920   46.575600   46.660324   14.350000   
...               ...        ...         ...         ...         ...   
2022-07-18   8.448000   8.400000  127.000000  128.020004  104.339996   
2022-07-19   8.648000   8.381000  129.059998  125.500000  105.839996   
2022-07-20   8.710000   8.750000  128.600006  129.300003  104.660004   
2022-07-21   8.613000   8.681000  128.740005  128.660004  104.559998   
2022-07-22   8.513000   8.560000  129.339996  128.139999  105.580002   

                           ALO.PA                 BN.PA             ...  \
                  Open      Close       Open      Close       Open  ...   
Date                                                                ...   
2010-01-04   14.000000  51.520000  49.455002  43.220001  42.724991  ...   
2010-01-05   13.880000  51.660000  51.520000  42.845001  43.099998  ...   
2010-01-06   13.960000  51.970001  51.950001  42.840000  42.840000  ...   
2010-01-07   13.910000  52.130001  51.799999  42.750000  42.695000  ...   
2010-01-08   14.315000  53.150002  52.410000  42.650002  42.849998  ...   
...                ...        ...        ...        ...        ...  ...   
2022-07-18  104.519997  24.790001  23.969999  53.520000  53.830002  ...   
2022-07-19  103.000000  24.139999  23.959999  54.619999  53.669998  ...   
2022-07-20  106.360001  23.240000  24.049999  54.020000  54.980000  ...   
2022-07-21  104.580002  23.000000  23.200001  53.830002  54.000000  ...   
2022-07-22  104.120003  22.860001  22.969999  53.740002  53.889999  ...   

               TTE.PA                 URW.AS                 VIE.PA  \
                Close       Open       Close        Open      Close   
Date                                                                  
2010-01-04  45.794991  45.369999  155.300003  152.899994  23.350000   
2010-01-05  45.849998  45.759998  155.100006  154.850006  23.530001   
2010-01-06  45.880001  45.674999  156.449997  155.800003  23.875000   
2010-01-07  45.959999  45.764999  154.800003  156.149994  24.155001   
2010-01-08  46.009998  46.009998  154.850006  155.800003  25.100000   
...               ...        ...         ...         ...        ...   
2022-07-18  48.645000  48.200001   49.369999   49.244999  23.430000   
2022-07-19  49.099998  48.480000   50.430000   48.770000  24.240000   
2022-07-20  49.200001  49.805000   49.849998   50.590000  23.760000   
2022-07-21  47.884998  49.325001   48.959999   49.810001  23.709999   
2022-07-22  48.540001  47.820000   52.020000   48.994999  23.770000   

                          VIV.PA                WLN.PA             
                 Open      Close       Open      Close       Open  
Date                                                               
2010-01-04  23.254999  20.719360  20.220974        NaN        NaN  
2010-01-05  23.365000  20.714521  20.758072        NaN        NaN  
2010-01-06  23.549999  20.491941  20.748392        NaN        NaN  
2010-01-07  23.840000  20.429037  20.516134        NaN        NaN  
2010-01-08  25.500000  20.187103  20.303232        NaN        NaN  
...               ...        ...        ...        ...        ...  
2022-07-18  22.990000   9.770000   9.636000  36.250000  35.669998  
2022-07-19  23.260000   9.856000   9.762000  37.220001  35.860001  
2022-07-20  24.400000  10.010000   9.900000  37.360001  37.650002  
2022-07-21  23.760000   9.996000   9.948000  37.310001  37.060001  
2022-07-22  23.700001  10.010000   9.964000  36.590000  37.130001  

[3217 r

In [99]:
cac.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
ACA.PA,Crédit Agricole,26.134925,0.026803,-0.289926,True,True,True
AI.PA,Air Liquide,1.590601,1.693049,1.754609,False,False,False
AIR.PA,Airbus,3.989348,1.882469,6.541429,True,False,False
ALO.PA,Alstom,6.209326,0.072308,-0.537762,True,True,True
BN.PA,Danone,1.135094,1.086529,0.257812,True,True,True
BNP.PA,BNP Paribas,13.443047,0.056714,-0.219123,True,True,True
CA.PA,Carrefour,1.385235,0.347489,-0.501036,True,True,True
CAP.PA,Capgemini,13.911497,0.380294,4.346875,True,True,True
CS.PA,AXA,3.178377,0.392889,0.280797,True,True,True
DG.PA,Vinci,1.373168,1.610380,1.257878,False,True,False


In [100]:
cac.top_quint

,Security
MT.AS,ArcelorMittal
RNO.PA,Renault
ACA.PA,Crédit Agricole
GLE.PA,Société Générale
CAP.PA,Capgemini


In [101]:
time.sleep(180)

## FTSE 100

In [102]:
ftse = OvI(ftse_df)
ftse.data

[*********************100%***********************]  100 of 100 completed


AAF.L                    AAL.L                   ABDN.L  \
                 Close        Open        Close         Open       Close   
Date                                                                       
2010-01-04         NaN         NaN  2756.736328  2683.077393  295.847504   
2010-01-05         NaN         NaN  2802.026611  2778.137207  302.552307   
2010-01-06         NaN         NaN  2850.302979  2816.957520  301.714203   
2010-01-07         NaN         NaN  2830.892822  2849.805420  294.031586   
2010-01-08         NaN         NaN  2883.648682  2835.869873  298.780792   
...                ...         ...          ...          ...         ...   
2022-07-18  159.600006  157.899994  2637.500000  2620.000000  155.199997   
2022-07-19  163.399994  161.100006  2637.000000  2574.000000  158.699997   
2022-07-20  164.699997  163.699997  2604.000000  2670.500000  158.250000   
2022-07-21  165.399994  166.899994  2627.500000  2606.500000  161.149994   
2022-07-22  164.699997  166.699997  2665.500000  2626.500000  161.600006   

                         ABF.L           ADM.L          ...        UTG.L  \
                  Open   Close    Open   Close    Open  ...        Close   
Date                                                    ...                
2010-01-04  296.545898   822.0   819.5  1174.0  1183.0  ...   301.899994   
2010-01-05  294.590393   821.0   821.5  1183.0  1177.0  ...   308.399994   
2010-01-06  299.479187   820.0   821.0  1163.0  1184.0  ...   307.000000   
2010-01-07  300.876099   838.5   819.5  1154.0  1160.0  ...   300.799988   
2010-01-08  296.266602   850.0   839.0  1149.0  1152.0  ...   296.100006   
...                ...     ...     ...     ...     ...  ...          ...   
2022-07-18  154.500000  1618.5  1607.5  1738.0  1806.5  ...  1146.000000   
2022-07-19  154.000000  1667.0  1615.5  1729.0  1739.0  ...  1166.000000   
2022-07-20  159.699997  1678.0  1672.0  1749.0  1724.0  ...  1161.000000   
2022-07-21  159.500000  1691.0  1696.5  1745.5  1747.5  ...  1172.000000   
2022-07-22  160.600006  1699.0  1689.5  1735.5  1752.0  ...  1207.000000   

                                UU.L                    VOD.L              \
                   Open        Close         Open       Close        Open   
Date                                                                        
2010-01-04   296.200012   496.299988   500.000000  149.409500  149.722198   
2010-01-05   301.700012   494.500000   497.100006  149.200897  149.096695   
2010-01-06   304.299988   496.500000   495.600006  148.575302  149.513702   
2010-01-07   309.000000   489.100006   494.899994  144.717606  147.011398   
2010-01-08   300.299988   496.000000   489.899994  142.840897  144.821899   
...                 ...          ...          ...         ...         ...   
2022-07-18  1160.000000  1041.500000  1015.000000  130.119995  128.919998   
2022-07-19  1147.000000  1045.500000  1047.000000  131.000000  129.779999   
2022-07-20  1174.000000  1055.000000  1050.000000  129.259995  131.580002   
2022-07-21  1171.000000  1058.500000  1059.000000  129.600006  128.419998   
2022-07-22  1189.000000  1077.500000  1059.000000  129.020004  129.119995   

                 WPP.L                    WTB.L               
                 Close        Open        Close         Open  
Date                                                          
2010-01-04  617.000000  602.000000  1202.771851  1189.885010  
2010-01-05  606.500000  614.500000  1201.053589  1206.208252  
2010-01-06  611.500000  606.500000  1198.476196  1203.630859  
2010-01-07  615.500000  610.500000  1199.335327  1193.321533  
2010-01-08  626.000000  617.500000  1212.222168  1200.194458  
...                ...         ...          ...          ...  
2022-07-18  811.000000  802.799988  2655.000000  2636.000000  
2022-07-19  835.200012  809.000000  2681.000000  2619.000000  
2022-07-20  853.799988  851.000000  2695.000000  2689.000000  
2022-07-21  869.799988  860.599976  2667.000000  269

In [103]:
ftse.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
AAF.L,Airtel Africa,24.608027,0.084754,1.138961,True,True,True
AAL.L,Anglo American plc,7.046784,0.140638,-0.006551,True,True,True
ABDN.L,Abrdn,1.293527,0.421283,-0.455059,True,True,True
ABF.L,Associated British Foods,2.970858,0.697851,1.073215,True,True,True
ADM.L,Admiral Group,3.182825,0.460922,0.467033,True,True,True
...,...,...,...,...,...,...,...
UTG.L,Unite Group,3.865132,1.054285,3.074949,True,True,True
UU.L,United Utilities,6.518195,0.330613,1.155000,True,True,True
VOD.L,Vodafone Group,6.551653,0.131529,-0.138271,True,True,True
WPP.L,WPP plc,1.546479,0.940249,0.447840,True,True,True


In [104]:
ftse.top_quint

,Security
JD.L,JD Sports
AAF.L,Airtel Africa
SMT.L,Scottish Mortgage Investment Trust
AHT.L,Ashtead Group
FRES.L,Fresnillo plc
TW.L,Taylor Wimpey
ICP.L,Intermediate Capital Group
IAG.L,International Airlines Group
SGRO.L,Segro
HWDN.L,Howdens Joinery


In [105]:
time.sleep(180)

## FTSE MIB

In [106]:
mib = OvI(mib_df)
mib.data

[*********************100%***********************]  40 of 40 completed


A2A.MI             AMP.MI                ATL.MI             \
             Close    Open      Close       Open      Close       Open   
Date                                                                     
2010-01-04  1.4690  1.4770   3.027500   3.055000  18.469999  18.360001   
2010-01-05  1.4600  1.4690   3.015000   3.017500  18.610001  18.469999   
2010-01-06  1.4480  1.4540   3.035000   3.027500  18.660000  18.650000   
2010-01-07  1.4280  1.4450   3.000000   3.035000  18.570000  18.600000   
2010-01-08  1.4230  1.4230   3.002500   3.040000  18.570000  18.660000   
...            ...     ...        ...        ...        ...        ...   
2022-07-18  1.1930  1.1865  29.799999  29.459999  22.360001  22.320000   
2022-07-19  1.2240  1.1840  29.799999  29.350000  22.480000  22.320000   
2022-07-20  1.2035  1.2390  29.780001  29.750000  22.450001  22.490000   
2022-07-21  1.2090  1.1760  30.660000  29.059999  22.510000  22.370001   
2022-07-22  1.2120  1.2110  31.160000  30.530001  22.540001  22.500000   

               AZM.MI               BAMI.MI             ...  TEN.MI          \
                Close       Open      Close       Open  ...   Close    Open   
Date                                                    ...                   
2010-01-04   9.049584   8.582685  22.371658  21.616409  ...  15.420  15.120   
2010-01-05   9.086203   9.090780  22.596191  22.453306  ...  15.610  15.470   
2010-01-06   8.999232   9.086203  22.800312  22.494129  ...  15.700  15.690   
2010-01-07   9.090780   9.008387  23.106493  22.657427  ...  15.820  15.650   
2010-01-08   9.026696   9.109090  23.024845  23.024845  ...  15.880  15.910   
...               ...        ...        ...        ...  ...     ...     ...   
2022-07-18  16.379999  16.350000   2.389000   2.330000  ...  12.685  12.285   
2022-07-19  16.900000  16.230000   2.512000   2.367000  ...  12.725  12.630   
2022-07-20  16.764999  16.799999   2.453000   2.528000  ...  12.595  12.780   
2022-07-21  16.240000  16.100000   2.354000   2.350000  ...  12.350  12.565   
2022-07-22  16.320000  16.160000   2.346000   2.348000  ...  12.490  12.375   

            TIT.MI          TRN.MI             UCG.MI                UNI.MI  \
             Close    Open   Close    Open      Close       Open      Close   
Date                                                                          
2010-01-04  1.0860  1.0880  2.9775  3.0000  75.357376  74.726768  17.185850   
2010-01-05  1.0930  1.0900  2.9550  2.9725  74.253815  75.278549  17.168104   
2010-01-06  1.0910  1.0980  2.9400  2.9575  73.623207  74.411469  17.318935   
2010-01-07  1.0790  1.0890  2.9350  2.9375  75.357376  73.544380  17.301191   
2010-01-08  1.0660  1.0840  2.9375  2.9475  76.066811  76.224457  17.230211   
...            ...     ...     ...     ...        ...        ...        ...   
2022-07-18  0.2260  0.2260  7.2340  7.0340   8.717000   8.660000   4.022000   
2022-07-19  0.2290  0.2272  7.2680  7.1860   9.266000   8.700000   4.075000   
2022-07-20  0.2308  0.2303  7.0920  7.2640   9.009000   9.380000   4.002000   
2022-07-21  0.2203  0.2228  7.0260  6.9300   8.705000   8.495000   3.856000   
2022-07-22  0.2158  0.2200  7.1980  7.0140   8.500000   8.610000   3.849000   

                       
                 Open  
Date                   
2010-01-04  17.070509  
2010-01-05  17.176977  
2010-01-06  17.239084  
2010-01-07  17.212467  
2010-01-08  17.372169  
...               ...  
2022-07-18   4.000000  
2022-07-19   3.990000  
2022-07-20   4.095000  
2022-07-21   3.855000  
2022-07-22   3.856000  

[3192 rows x 80 columns]

In [107]:
mib.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
A2A.MI,A2A,9.516471,0.086228,-0.179418,True,True,True
AMP.MI,Amplifon,8.782842,1.161318,9.199672,True,False,False
ATL.MI,Atlantia,4.462400,0.275114,0.227669,True,True,True
AZM.MI,Azimut,48.325823,0.039348,0.901503,True,True,True
BAMI.MI,Banco BPM,122.467558,0.000886,-0.891471,True,True,True
BGN.MI,Banca Generali,23.827459,0.135024,2.217288,True,True,True
BMED.MI,Banca Mediolanum,9.788590,0.146389,0.432942,True,True,True
BPE.MI,BPER Banca,17.740363,0.007765,-0.862250,True,True,True
CNHI.MI,CNH Industrials N.V.,0.493097,2.467614,0.216772,False,True,False
CPR.MI,Campari,2.474946,2.306023,4.707284,True,False,False


In [108]:
mib.top_quint

,Security
BAMI.MI,Banco BPM
AZM.MI,Azimut
UNI.MI,Unipol
STLA.MI,Stellantis
BGN.MI,Banca Generali
TIT.MI,Telecom Italia
UCG.MI,UniCredit


In [109]:
time.sleep(180)

## IBEX 35

In [110]:
ibex = OvI(ibex_df)
ibex.data

[*********************100%***********************]  33 of 33 completed


ACS.MC             ACX.MC             AENA.MC              \
                Close       Open   Close    Open       Close        Open   
Date                                                                       
2010-01-02        NaN        NaN     NaN     NaN         NaN         NaN   
2010-01-04  35.860001  35.070000  14.870  14.570         NaN         NaN   
2010-01-05  36.490002  37.650002  14.985  14.965         NaN         NaN   
2010-01-06  36.564999  36.490002  14.920  15.000         NaN         NaN   
2010-01-07  36.200001  36.279999  15.000  14.815         NaN         NaN   
...               ...        ...     ...     ...         ...         ...   
2022-07-18  21.770000  21.850000   8.620   8.570  122.800003  123.699997   
2022-07-19  22.020000  21.660000   8.830   8.612  125.550003  122.550003   
2022-07-20  21.780001  22.170000   8.908   8.880  123.599998  126.000000   
2022-07-21  21.780001  21.700001   8.878   8.892  123.949997  122.800003   
2022-07-22  22.219999  21.760000   8.896   8.826  125.500000  124.000000   

               AMS.MC                 ANA.MC              ...    SAB.MC  \
                Close       Open       Close        Open  ...     Close   
Date                                                      ...             
2010-01-02        NaN        NaN         NaN         NaN  ...       NaN   
2010-01-04        NaN        NaN   94.389999   91.910004  ...  2.347240   
2010-01-05        NaN        NaN   93.500000   94.339996  ...  2.382805   
2010-01-06        NaN        NaN   93.650002   92.750000  ...  2.430224   
2010-01-07        NaN        NaN   93.150002   93.000000  ...  2.427853   
...               ...        ...         ...         ...  ...       ...   
2022-07-18  55.060001  55.160000  183.100006  183.300003  ...  0.621400   
2022-07-19  56.200001  54.500000  183.000000  182.199997  ...  0.660800   
2022-07-20  55.900002  56.700001  179.000000  181.199997  ...  0.644800   
2022-07-21  55.560001  55.599998  180.600006  178.800003  ...  0.661800   
2022-07-22  56.000000  55.540001  184.699997  181.100006  ...  0.630000   

                         SAN.MC               SGRE.MC                SLR.MC  \
                Open      Close       Open      Close       Open      Close   
Date                                                                          
2010-01-02       NaN        NaN        NaN        NaN        NaN        NaN   
2010-01-04  2.314640  10.916778  10.740401  12.235000  11.910000   2.640000   
2010-01-05  2.367986  11.037456  10.926061  12.470000  12.370000   2.730000   
2010-01-06  2.388732  11.116361  11.037456  12.575000  12.470000   2.730000   
2010-01-07  2.430224  11.065305  11.111720  12.630000  12.500000   2.710000   
...              ...        ...        ...        ...        ...        ...   
2022-07-18  0.635000   2.412000   2.428000  17.975000  17.910000  20.840000   
2022-07-19  0.621200   2.512500   2.401500  17.945000  17.950001  20.559999   
2022-07-20  0.669600   2.461000   2.540000  17.985001  17.940001  20.330000   
2022-07-21  0.641400   2.453500   2.436000  17.920000  17.975000  20.620001   
2022-07-22  0.653000   2.433000   2.455000  17.910000  17.930000  21.629999   

                          TEF.MC             
                 Open      Close       Open  
Date                                         
2010-01-02        NaN        NaN        NaN  
2010-01-04   2.575000  19.595512  19.353289  
2010-01-05   2.660000  19.565853  19.595512  
2010-01-06   2.750000  19.422495  19.565853  
2010-01-07   2.700000  19.229704  19.387892  
...               ...        ...        ...  
2022-07-18  20.690001   4.613000   4.658000  
2022-07-19  20.860001   4.612000   4.599000  
2022-07-20  20.350000   4.568000   4.618000  
2022-07-21  20.139999   4.462000   4.530000  
2022-07-22  20.809999   4.385000   4.457000  

[3216 rows x 66 columns]

In [111]:
ibex.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
ACS.MC,ACS,5.412528,0.117060,-0.366410,True,True,True
ACX.MC,Acerinox,45.727107,0.013762,-0.389430,True,True,True
AENA.MC,Aena,3.867508,0.498461,0.927803,True,True,True
AMS.MC,Amadeus IT Group,5.798436,0.887731,4.090909,True,True,True
ANA.MC,Acciona,10.151135,0.196602,1.009574,True,True,True
BBVA.MC,BBVA,1.873923,0.180988,-0.660842,True,True,True
BKT.MC,Bankinter,6.822937,0.152443,0.006312,True,True,True
CABK.MC,CaixaBank,5.501561,0.153713,-0.149141,True,True,True
CLNX.MC,Cellnex Telecom,6.566350,0.472349,2.101611,True,True,True
COL.MC,Inmobiliaria Colonial,27.377854,0.004660,-0.870683,True,True,True


In [112]:
ibex.top_quint

,Security
SLR.MC,Solaria
PHM.MC,PharmaMar
MTS.MC,ArcelorMittal
ACX.MC,Acerinox
IAG.MC,IAG
MEL.MC,Meliá Hotels


In [113]:
time.sleep(180)

## Nikkei 225

In [114]:
nikkei = OvI(nikkei_df)
nikkei.data

[*********************100%***********************]  225 of 225 completed


1332.T         1333.T          1605.T          1721.T          \
            Close   Open   Close    Open   Close    Open   Close    Open   
Date                                                                       
2010-01-04  261.0  262.0     NaN     NaN  1787.5  1757.5   983.0   987.0   
2010-01-05  260.0  261.0     NaN     NaN  1820.0  1822.5   974.0   984.0   
2010-01-06  260.0  261.0     NaN     NaN  1830.0  1812.5   990.0   974.0   
2010-01-07  258.0  259.0     NaN     NaN  1832.5  1850.0   979.0   975.0   
2010-01-08  264.0  262.0     NaN     NaN  1792.5  1837.5   996.0   984.0   
...           ...    ...     ...     ...     ...     ...     ...     ...   
2022-07-15  577.0  588.0  2478.0  2502.0  1346.0  1375.0  2563.0  2569.0   
2022-07-19  584.0  582.0  2467.0  2465.0  1409.0  1405.0  2549.0  2575.0   
2022-07-20  591.0  586.0  2475.0  2488.0  1422.0  1433.0  2609.0  2565.0   
2022-07-21  598.0  591.0  2481.0  2456.0  1416.0  1435.0  2635.0  2603.0   
2022-07-22  592.0  593.0  2458.0  2477.0  1400.0  1400.0  2652.0  2651.0   

            1801.T          ...       9613.T               9735.T          \
             Close    Open  ...        Close         Open   Close    Open   
Date                        ...                                             
2010-01-04   800.0   805.0  ...   581.799988   577.599976  4460.0  4410.0   
2010-01-05   815.0   815.0  ...   596.200012   590.200012  4460.0  4530.0   
2010-01-06   845.0   815.0  ...   603.000000   600.000000  4480.0  4465.0   
2010-01-07   845.0   850.0  ...   595.000000   593.000000  4430.0  4445.0   
2010-01-08   900.0   875.0  ...   577.799988   598.000000  4355.0  4455.0   
...            ...     ...  ...          ...          ...     ...     ...   
2022-07-15  4085.0  4115.0  ...  1866.000000  1875.000000  8871.0  8971.0   
2022-07-19  4125.0  4125.0  ...  1885.000000  1881.000000  8818.0  8970.0   
2022-07-20  4205.0  4190.0  ...  1939.000000  1907.000000  8923.0  8886.0   
2022-07-21  4205.0  4150.0  ...  1980.000000  1952.000000  8977.0  8864.0   
2022-07-22  4225.0  4205.0  ...  1996.000000  1990.000000  8985.0  9006.0   

            9766.T           9983.T           9984.T          
             Close    Open    Close     Open   Close    Open  
Date                                                          
2010-01-04  1660.0  1663.0  17460.0  17660.0  1088.0  1085.0  
2010-01-05  1713.0  1700.0  16900.0  17210.0  1090.0  1096.5  
2010-01-06  1712.0  1720.0  16770.0  16960.0  1094.0  1090.0  
2010-01-07  1756.0  1725.0  16690.0  16780.0  1098.0  1091.5  
2010-01-08  1718.0  1725.0  16900.0  16780.0  1090.0  1099.0  
...            ...     ...      ...      ...     ...     ...  
2022-07-15  7380.0  7410.0  76230.0  73730.0  5296.0  5361.0  
2022-07-19  7310.0  7390.0  77540.0  78130.0  5371.0  5348.0  
2022-07-20  7450.0  7350.0  79550.0  78360.0  5500.0  5490.0  
2022-07-21  7550.0  7510.0  80020.0  79360.0  5507.0  5500.0  
2022-07-22  7780.0  7700.0  80210.0  80010.0  5482.0  5503.0  

[3092 rows x 450 columns]

In [115]:
nikkei.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
1332.T,"NIPPON SUISAN KAISHA, LTD.",4.062430,0.550202,1.259542,True,True,True
1333.T,MARUHA NICHIRO CORP.,1.277462,1.144632,0.462225,True,True,True
1605.T,INPEX CORP.,8.101039,0.098691,-0.203414,True,True,True
1721.T,COMSYS HOLDINGS CORP.,1.299759,1.953870,1.686930,False,False,False
1801.T,TAISEI CORP.,3.777397,1.357857,4.248447,True,False,False
...,...,...,...,...,...,...,...
9613.T,NTT DATA CORP.,2.164239,1.555108,2.455679,True,False,False
9735.T,"SECOM CO., LTD.",5.449200,0.362224,1.037415,True,True,True
9766.T,KONAMI GROUP CORP.,3.739859,1.223425,3.678292,True,True,True
9983.T,"FAST RETAILING CO., LTD.",4.872042,0.908893,3.541903,True,True,True


In [116]:
nikkei.top_quint

,Security
6861.T,KEYENCE CORP.
4063.T,"SHIN-ETSU CHEMICAL CO., LTD."
7733.T,OLYMPUS CORP.
1808.T,HASEKO CORP.
6479.T,MINEBEA MITSUMI INC.
5703.T,"NIPPON LIGHT METAL HOLDINGS CO., LTD."
1925.T,"DAIWA HOUSE IND. CO., LTD."
6762.T,TDK CORP.
6367.T,"DAIKIN INDUSTRIES, LTD."
8591.T,ORIX CORP.


In [117]:
time.sleep(180)

## KOSPI Composite

In [118]:
kospi = OvI(kospi_df)
kospi.data

[*********************100%***********************]  824 of 824 completed

6 Failed downloads:
- 015350.KS: No data found, symbol may be delisted
- 138250.KS: No data found, symbol may be delisted
- 101060.KS: No data found, symbol may be delisted
- 153360.KS: No data found, symbol may be delisted
- 172580.KS: No data found, symbol may be delisted
- 007630.KS: No data found, symbol may be delisted


000020.KS             000040.KS                  000050.KS  \
               Close     Open        Close         Open         Close   
Date                                                                    
2010-01-04    7520.0   7540.0  3569.998291  3359.998291   9886.363281   
2010-01-05    7500.0   7490.0  3539.998291  3569.998291   9924.242188   
2010-01-06    7350.0   7500.0  3629.998291  3629.998291  10189.393555   
2010-01-07    7210.0   7350.0  3629.998291  3629.998291  10151.515625   
2010-01-08    7240.0   7220.0  3719.998047  3659.998291  10000.000000   
...              ...      ...          ...          ...           ...   
2022-07-18   11000.0  11100.0   742.000000   775.000000  12700.000000   
2022-07-19   11100.0  11050.0   763.000000   737.000000  12650.000000   
2022-07-20   11150.0  11250.0   756.000000   763.000000  12800.000000   
2022-07-21   11050.0  11200.0   753.000000   758.000000  12900.000000   
2022-07-22   10800.0  11050.0   743.000000   755.000000  12800.000000   

                         000060.KS              000070.KS                ...  \
                    Open     Close     Open         Close          Open  ...   
Date                                                                     ...   
2010-01-04   9848.484375    7200.0   6900.0  46516.675781  46309.011719  ...   
2010-01-05   9924.242188    7340.0   7230.0  46257.097656  46101.347656  ...   
2010-01-06   9924.242188    7560.0   7400.0  45893.683594  46257.097656  ...   
2010-01-07   9962.121094    7590.0   7560.0  45166.859375  45841.769531  ...   
2010-01-08   9962.121094    7500.0   7520.0  45063.031250  45478.355469  ...   
...                  ...       ...      ...           ...           ...  ...   
2022-07-18  12600.000000   31200.0  30850.0  72600.000000  72800.000000  ...   
2022-07-19  12700.000000   33300.0  31200.0  72900.000000  71200.000000  ...   
2022-07-20  12700.000000   33750.0  33500.0  73900.000000  73400.000000  ...   
2022-07-21  12800.000000   33200.0  33250.0  72700.000000  73700.000000  ...   
2022-07-22  12950.000000   32850.0  32750.0  72200.000000  72500.000000  ...   

           400760.KS         402340.KS          404990.KS         900140.KS  \
               Close    Open     Close     Open     Close    Open     Close   
Date                                                                          
2010-01-04       NaN     NaN       NaN      NaN       NaN     NaN       NaN   
2010-01-05       NaN     NaN       NaN      NaN       NaN     NaN       NaN   
2010-01-06       NaN     NaN       NaN      NaN       NaN     NaN       NaN   
2010-01-07       NaN     NaN       NaN      NaN       NaN     NaN       NaN   
2010-01-08       NaN     NaN       NaN      NaN       NaN     NaN       NaN   
...              ...     ...       ...      ...       ...     ...       ...   
2022-07-18    4450.0  4460.0   40950.0  40650.0    4990.0  5000.0    2970.0   
2022-07-19    4395.0  4450.0   40400.0  40850.0    4990.0  4965.0    2925.0   
2022-07-20    4435.0  4395.0   41500.0  41400.0    4825.0  4995.0    2925.0   
2022-07-21    4410.0  4435.0   41750.0  41600.0    4850.0  4835.0    2920.0   
2022-07-22    4380.0  4385.0   41150.0  41350.0    4805.0  4850.0    2890.0   

                   950210.KS           
              Open     Close     Open  
Date                                   
2010-01-04     NaN       NaN      NaN  
2010-01-05     NaN       NaN      NaN  
2010-01-06     NaN       NaN      NaN  
2010-01-07     NaN       NaN      NaN  
2010-01-08     NaN       NaN      NaN  
...            ...       ...      ...  
2022-07-18  2995.0   12650.0  13100.0  
2022-07-19  2970.0   12300.0  12400.0  
2022-07-20  2950.0   12600.0  12400.0  
2022-07-21  2975.0   12250.0  12500.0  
2022-07-22  2930.0   12000.0  12250.0  

[3098 rows x 1636 columns]

In [119]:
kospi.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
000020.KS,DONGWHA PHARM,58.612832,0.024104,0.432361,True,True,True
000040.KS,KR MOTORS,21.278216,0.010383,-0.778869,True,True,True
000050.KS,KYUNGBANG,1.501097,0.851693,0.299692,True,True,True
000060.KS,Meritz Insurance,0.899769,5.235138,3.760870,False,False,False
000070.KS,Samyang Holdings,0.581147,2.653881,0.559092,False,True,False
...,...,...,...,...,...,...,...
400760.KS,NH All-One REIT,0.881659,0.923403,-0.185874,False,True,False
402340.KS,SKSQUARE,0.937794,0.535117,-0.498171,True,True,True
404990.KS,SHSBREIT,0.911684,1.096871,0.000000,False,True,False
900140.KS,LVMC,1.028896,0.458229,-0.528530,True,True,True


In [120]:
kospi.top_quint

,Security
011000.KS,GENEONE
001880.KS,DL Con
012690.KS,MONALISA
008970.KS,DONGYANG STL PIPE
101140.KS,INBIOGEN
...,...
007810.KS,KOR CIRC
042670.KS,HDSINFRA
066570.KS,LG ELECTRONICS INC.
007120.KS,MiraeING


In [121]:
time.sleep(180)

## TSEC Weighted

In [122]:
tsec = OvI(tsec_df)
tsec.data

[*********************100%***********************]  753 of 753 completed

6 Failed downloads:
- 3092.TWO: No data found, symbol may be delisted
- 1752.TWO: No data found, symbol may be delisted
- 5306.TWO: No data found, symbol may be delisted
- 3144.TWO: No data found, symbol may be delisted
- 4152.TWO: No data found, symbol may be delisted
- 6426.TWO: No data found, symbol may be delisted


1240.TWO              1259.TWO            1264.TWO         \
                Close       Open      Close       Open    Close   Open   
Date                                                                     
2010-01-04        NaN        NaN        NaN        NaN      NaN    NaN   
2010-01-05        NaN        NaN        NaN        NaN      NaN    NaN   
2010-01-06        NaN        NaN        NaN        NaN      NaN    NaN   
2010-01-07        NaN        NaN        NaN        NaN      NaN    NaN   
2010-01-08        NaN        NaN        NaN        NaN      NaN    NaN   
...               ...        ...        ...        ...      ...    ...   
2022-07-18  45.299999  45.049999  68.199997  67.199997    258.5  258.5   
2022-07-19  46.150002  45.849998  68.500000  68.400002    258.5  258.5   
2022-07-20  46.150002  47.200001  68.599998  69.400002    260.0  257.0   
2022-07-21  46.750000  46.299999  69.500000  68.699997    258.5  259.5   
2022-07-22  46.549999  46.400002  69.800003  69.800003    258.5  257.5   

           1268.TWO          1336.TWO             ...   9949.TWO             \
              Close   Open      Close       Open  ...      Close       Open   
Date                                              ...                         
2010-01-04      NaN    NaN  20.100000  20.299999  ...  29.996319  30.318365   
2010-01-05      NaN    NaN  19.900000  20.000000  ...  29.904306  29.996319   
2010-01-06      NaN    NaN  19.600000  19.600000  ...  29.766287  29.904306   
2010-01-07      NaN    NaN  18.700001  19.600000  ...  29.168200  29.720280   
2010-01-08      NaN    NaN  19.200001  18.799999  ...  28.340080  28.984173   
...             ...    ...        ...        ...  ...        ...        ...   
2022-07-18    107.5  105.0  20.299999  20.000000  ...  20.000000  19.850000   
2022-07-19    109.0  107.5  20.150000  20.000000  ...  19.650000  19.600000   
2022-07-20    107.5  109.5  20.600000  20.100000  ...  19.900000  19.900000   
2022-07-21    107.5  108.0  20.600000  20.650000  ...  19.900000  19.950001   
2022-07-22    107.0  107.0  21.049999  20.600000  ...  19.750000  19.400000   

             9950.TWO              9951.TWO              9960.TWO             \
                Close       Open      Close       Open      Close       Open   
Date                                                                           
2010-01-04  15.278494  15.109203  33.150002  33.500000  21.473562  21.771807   
2010-01-05  15.405462  15.320816  32.299999  33.200001  21.622684  21.622684   
2010-01-06  15.786366  15.320816  32.150002  32.250000  23.113905  21.548124   
2010-01-07  15.913334  15.828689  31.900000  32.150002  22.964783  22.964783   
2010-01-08  15.363139  15.236171  32.099998  31.700001  23.151184  23.113905   
...               ...        ...        ...        ...        ...        ...   
2022-07-18  12.500000  12.550000  62.200001  61.000000  26.750000  26.549999   
2022-07-19  12.500000  12.500000  63.700001  62.299999  26.650000  26.700001   
2022-07-20  12.150000  11.400000  64.199997  64.000000  26.700001  26.450001   
2022-07-21  12.150000  12.150000  63.700001  65.599998  26.700001  26.500000   
2022-07-22  12.500000  12.500000  63.299999  63.299999  26.700001  26.600000   

             9962.TWO             
                Close       Open  
Date                              
2010-01-04  13.165379  13.652986  
2010-01-05  13.002844  13.327915  
2010-01-06  13.002844  12.840308  
2010-01-07  13.002844  13.084112  
2010-01-08  12.921576  13.165379  
...               ...        ...  
2022-07-18  13.500000  13.050000  
2022-07-19  14.300000  13.550000  
2022-07-20  14.450000  14.550000  
2022-07-21  14.950000  14.450000  
2022-07-22  15.100000  15.000000  

[3081 rows x 1494 columns]

In [123]:
tsec.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
1240.TWO,MORNSUN,4.571876,0.228378,0.080295,True,True,True
1259.TWO,AN-SHIN,1.543033,0.292954,-0.537470,True,True,True
1264.TWO,TEHMAG,1.760588,0.622821,0.108095,True,True,True
1268.TWO,Hi-Lai Foods,0.709100,0.821490,-0.407132,False,True,False
1336.TWO,TAIHAN,4.269511,0.244785,0.036946,True,True,True
...,...,...,...,...,...,...,...
9949.TWO,TITTOT,0.000249,2688.395108,-0.348580,False,True,False
9950.TWO,EMINENT,0.328278,2.606414,-0.172690,False,True,False
9951.TWO,MACAUTO,283.509505,0.006249,0.889552,True,True,True
9960.TWO,DACOME,0.053436,22.213009,0.226357,False,False,False


In [124]:
tsec.top_quint

,Security
3687.TWO,OMG
6182.TWO,WAFER WORKS
3691.TWO,GIGASOLAR
6118.TWO,Xander
5483.TWO,SAS
...,...
3221.TWO,TST
3675.TWO,Eris Tech
5490.TWO,XAC
3491.TWO,UMT


In [125]:
time.sleep(180)

## Hang Seng

In [126]:
hs = OvI(hs_df)
hs.data

[*********************100%***********************]  69 of 69 completed


0001.HK               0002.HK              0003.HK            \
                Close       Open      Close       Open     Close      Open   
Date                                                                         
2010-01-04  71.471809  71.614754  52.549999  52.500000  6.868090  6.819133   
2010-01-05  72.472420  72.400948  52.750000  52.650002  6.889072  6.889072   
2010-01-06  72.543892  73.401550  52.799999  52.849998  6.882078  6.889072   
2010-01-07  72.758301  72.543892  52.849998  52.900002  6.896066  6.924042   
2010-01-08  72.972717  72.186531  52.950001  52.750000  6.917048  6.882078   
...               ...        ...        ...        ...       ...       ...   
2022-07-18  53.450001  52.950001  66.900002  66.849998  8.350000  8.260000   
2022-07-19  52.900002  53.450001  66.250000  66.949997  8.270000  8.260000   
2022-07-20  52.950001  53.049999  66.250000  66.199997  8.260000  8.280000   
2022-07-21  52.400002  52.700001  65.849998  66.250000  8.210000  8.250000   
2022-07-22  52.099998  52.849998  65.699997  66.000000  8.210000  8.210000   

              0005.HK               0006.HK             ...    6862.HK  \
                Close       Open      Close       Open  ...      Close   
Date                                                    ...              
2010-01-04  89.250000  89.400002  42.549999  42.250000  ...        NaN   
2010-01-05  90.449997  90.199997  42.849998  43.000000  ...        NaN   
2010-01-06  91.599998  91.300003  42.650002  43.200001  ...        NaN   
2010-01-07  91.250000  91.699997  42.799999  42.950001  ...        NaN   
2010-01-08  91.699997  91.500000  42.900002  43.000000  ...        NaN   
...               ...        ...        ...        ...  ...        ...   
2022-07-18  49.150002  48.400002  51.150002  51.000000  ...  16.059999   
2022-07-19  48.799999  48.700001  50.650002  50.900002  ...  15.680000   
2022-07-20  49.250000  49.450001  50.799999  50.900002  ...  16.320000   
2022-07-21  48.200001  48.900002  50.700001  50.799999  ...  15.620000   
2022-07-22  48.599998  48.849998  51.299999  50.700001  ...  15.860000   

                          9618.HK                9633.HK             \
                 Open       Close        Open      Close       Open   
Date                                                                  
2010-01-04        NaN         NaN         NaN        NaN        NaN   
2010-01-05        NaN         NaN         NaN        NaN        NaN   
2010-01-06        NaN         NaN         NaN        NaN        NaN   
2010-01-07        NaN         NaN         NaN        NaN        NaN   
2010-01-08        NaN         NaN         NaN        NaN        NaN   
...               ...         ...         ...        ...        ...   
2022-07-18  16.020000  244.000000  238.399994  45.799999  45.849998   
2022-07-19  15.860000  241.000000  243.399994  45.950001  45.450001   
2022-07-20  15.800000  245.600006  247.399994  45.950001  46.099998   
2022-07-21  16.299999  247.399994  246.800003  45.900002  46.049999   
2022-07-22  15.860000  247.000000  250.000000  45.900002  45.799999   

               9988.HK                 9999.HK              
                 Close        Open       Close        Open  
Date                                                        
2010-01-04         NaN         NaN         NaN         NaN  
2010-01-05         NaN         NaN         NaN         NaN  
2010-01-06         NaN         NaN         NaN         NaN  
2010-01-07         NaN         NaN         NaN         NaN  
2010-01-08         NaN         NaN         NaN         NaN  
...                ...         ...         ...         ...  
2022-07-18  103.900002  100.800003  137.300003  134.699997  
2022-07-19  100.900002  101.800003  138.600006  137.800003  
2022-07-20  103.199997  104.800003  143.800003  141.199997  
2022-07-21  102.000000  102.800003  152.800003  148.100006  
2022-07-22  102.099998  102.699997  151.100006  156.199997  

[3099 rows x 138 columns]

In [127]:
hs.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
0001.HK,CKH HOLDINGS,3.223068,0.224098,-0.272496,True,True,True
0002.HK,CLP HOLDINGS,2.827104,0.442519,0.251429,True,True,True
0003.HK,HK & CHINA GAS,3.522150,0.328008,0.203965,True,True,True
0005.HK,HSBC HOLDINGS,0.077059,7.057451,-0.456376,False,True,False
0006.HK,POWER ASSETS,1.678794,0.721596,0.214201,True,True,True
...,...,...,...,...,...,...,...
6862.HK,HAIDILAO,6.413596,0.131536,-0.156383,True,True,True
9618.HK,JD-SW,0.422127,2.448251,0.033473,False,True,False
9633.HK,NONGFU SPRING,2.029881,0.568145,0.153266,True,True,True
9988.HK,BABA-SW,0.541870,1.007602,-0.454011,False,True,False


In [128]:
hs.top_quint

,Security
0881.HK,ZHONGSHENG HLDG
0669.HK,TECHTRONIC IND
0968.HK,XINYI SOLAR
1299.HK,AIA
0241.HK,ALI HEALTH
0960.HK,LONGFOR GROUP
2020.HK,ANTA SPORTS
2331.HK,LI NING
2688.HK,ENN ENERGY
0175.HK,GEELY AUTO


In [129]:
time.sleep(180)

## CSI 300 Index

In [130]:
csi = OvI(csi_df)
csi.data

[*********************100%***********************]  299 of 299 completed

6 Failed downloads:
- 600074.SS: No data found, symbol may be delisted
- 600317.SS: No data found, symbol may be delisted
- 600068.SS: No data found, symbol may be delisted
- 600485.SS: No data found, symbol may be delisted
- 002450.SZ: No data found, symbol may be delisted
- 300104.SZ: No data found, symbol may be delisted


000001.SZ             000002.SZ             000009.SZ             \
                Close       Open      Close       Open      Close       Open   
Date                                                                           
2010-01-04   8.575665   8.868634  10.600000  10.850000   4.848810   4.920379   
2010-01-05   8.427372   8.590133  10.360000  10.510000   4.723564   4.871175   
2010-01-06   8.282696   8.409288  10.360000  10.350000   4.719091   4.723564   
2010-01-07   8.192274   8.282696  10.280000  10.360000   4.808552   4.669887   
2010-01-08   8.174189   8.138020  10.350000  10.280000   5.041152   4.786187   
...               ...        ...        ...        ...        ...        ...   
2022-07-18  13.390000  13.250000  17.900000  17.740000  12.670000  12.250000   
2022-07-19  13.420000  13.340000  17.799999  17.940001  12.510000  12.760000   
2022-07-20  13.390000  13.430000  17.760000  17.879999  12.690000  12.590000   
2022-07-21  13.010000  13.320000  17.290001  17.650000  12.460000  12.400000   
2022-07-22  12.810000  12.840000  17.379999  17.299999  12.100000  12.510000   

           000027.SZ            000039.SZ             ... 601992.SS        \
               Close      Open      Close       Open  ...     Close  Open   
Date                                                  ...                   
2010-01-04  6.208333  6.375000  10.908333  11.041666  ...       NaN   NaN   
2010-01-05  6.203703  6.208333  11.058333  10.908333  ...       NaN   NaN   
2010-01-06  6.194444  6.212962  11.258333  11.100000  ...       NaN   NaN   
2010-01-07  6.055555  6.175925  11.083333  11.291666  ...       NaN   NaN   
2010-01-08  6.125000  6.032407  11.100000  11.000000  ...       NaN   NaN   
...              ...       ...        ...        ...  ...       ...   ...   
2022-07-18  6.810000  6.560000  13.210000  13.010000  ...      2.60  2.56   
2022-07-19  6.650000  6.820000  13.230000  13.180000  ...      2.61  2.61   
2022-07-20  6.670000  6.750000  13.420000  13.210000  ...      2.64  2.61   
2022-07-21  6.570000  6.670000  13.250000  13.410000  ...      2.62  2.64   
2022-07-22  6.490000  6.600000  13.330000  13.300000  ...      2.61  2.62   

           601998.SS           603000.SS         603885.SS             \
               Close      Open     Close   Open      Close       Open   
Date                                                                    
2010-01-04  7.667895  7.886435       NaN    NaN        NaN        NaN   
2010-01-05  7.762912  7.648892       NaN    NaN        NaN        NaN   
2010-01-06  7.544373  7.686899       NaN    NaN        NaN        NaN   
2010-01-07  7.287826  7.506366       NaN    NaN        NaN        NaN   
2010-01-08  7.354338  7.259321       NaN    NaN        NaN        NaN   
...              ...       ...       ...    ...        ...        ...   
2022-07-18  4.590000  4.560000     10.79  10.34  16.360001  16.200001   
2022-07-19  4.630000  4.590000     10.90  10.80  16.100000  16.340000   
2022-07-20  4.700000  4.630000     11.00  10.90  15.990000  16.280001   
2022-07-21  4.660000  4.690000     10.93  11.03  15.920000  15.990000   
2022-07-22  4.690000  4.660000     10.86  11.03  16.299999  15.970000   

           603993.SS        
               Close  Open  
Date                        
2010-01-04       NaN   NaN  
2010-01-05       NaN   NaN  
2010-01-06       NaN   NaN  
2010-01-07       NaN   NaN  
2010-01-08       NaN   NaN  
...              ...   ...  
2022-07-18      5.31  5.18  
2022-07-19      5.24  5.29  
2022-07-20      5.44  5.26  
2022-07-21      5.98  5.43  
2022-07-22      5.56  5.70  

[3051 rows x 586 columns]

In [131]:
csi.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
000001.SZ,Ping An Bank,0.054549,24.321813,0.444416,False,False,False
000002.SZ,China Vanke,0.030773,55.397405,0.601843,False,False,False
000009.SZ,China Baoan Group,1.605376,1.557114,1.459160,True,True,True
000027.SZ,Shenzhen Energy Group,0.792680,1.312695,0.018039,False,True,False
000039.SZ,China International Marine Containers,0.339333,3.729568,0.207245,False,True,False
...,...,...,...,...,...,...,...
601992.SS,BBMG Corporation,0.132602,2.765194,-0.652000,False,True,False
601998.SS,China Citic Bank,0.007964,77.457155,-0.405308,False,True,False
603000.SS,People.cn,0.038164,40.636640,0.400838,False,False,False
603885.SS,Juneyao Airlines,0.254365,11.341734,1.834782,False,False,False


In [132]:
csi.top_quint

,Security
000156.SZ,Wasu Media Holding


In [133]:
time.sleep(180)

## Straits Times Index

In [134]:
st = OvI(st_df)
st.data

[*********************100%***********************]  30 of 30 completed


9CI.SI         A17U.SI           AJBU.SI          BN4.SI            \
            Close  Open     Close      Open   Close  Open     Close      Open   
Date                                                                            
2010-01-04    NaN   NaN  2.072036  2.150227     NaN   NaN  7.259995  7.224881   
2010-01-05    NaN   NaN  2.062263  2.081810     NaN   NaN  7.356561  7.303889   
2010-01-06    NaN   NaN  2.052489  2.081810     NaN   NaN  7.391676  7.391676   
2010-01-07    NaN   NaN  2.013394  2.052489     NaN   NaN  7.628701  7.435570   
2010-01-08    NaN   NaN  1.984073  2.013394     NaN   NaN  7.593586  7.637480   
...           ...   ...       ...       ...     ...   ...       ...       ...   
2022-07-18   3.80  3.77  2.890000  2.880000    1.97  1.95  6.490000  6.420000   
2022-07-19   3.79  3.78  2.870000  2.870000    1.96  1.96  6.430000  6.460000   
2022-07-20   3.83  3.83  2.880000  2.890000    1.95  1.97  6.590000  6.500000   
2022-07-21   3.82  3.81  2.870000  2.880000    1.95  1.96  6.520000  6.600000   
2022-07-22   3.85  3.84  2.910000  2.880000    1.99  1.96  6.570000  6.560000   

           BS6.SI         ... U14.SI       U96.SI           V03.SI             \
            Close   Open  ...  Close  Open  Close  Open      Close       Open   
Date                      ...                                                   
2010-01-04  1.240  1.210  ...   4.11  4.07   3.71  3.69   8.780000   8.850000   
2010-01-05  1.260  1.250  ...   4.13  4.14   3.71  3.74   8.900000   8.870000   
2010-01-06  1.220  1.260  ...   4.14  4.14   3.74  3.73   9.090000   8.920000   
2010-01-07  1.250  1.220  ...   4.08  4.12   3.69  3.75   9.180000   9.130000   
2010-01-08  1.250  1.260  ...   4.07  4.08   3.70  3.70   9.200000   9.210000   
...           ...    ...  ...    ...   ...    ...   ...        ...        ...   
2022-07-18  0.890  0.895  ...   7.29  7.19   2.93  2.90  16.830000  16.780001   
2022-07-19  0.890  0.890  ...   7.26  7.30   3.05  2.98  16.700001  16.799999   
2022-07-20  0.905  0.890  ...   7.33  7.34   3.04  3.08  16.959999  16.900000   
2022-07-21  0.895  0.905  ...   7.31  7.35   2.97  3.04  17.090000  16.959999   
2022-07-22  0.915  0.895  ...   7.34  7.34   2.99  2.97  17.330000  17.090000   

           Y92.SI        Z74.SI        
            Close   Open  Close  Open  
Date                                   
2010-01-04  0.245  0.245   3.13  3.11  
2010-01-05  0.250  0.250   3.12  3.16  
2010-01-06  0.250  0.250   3.10  3.11  
2010-01-07  0.250  0.250   3.03  3.10  
2010-01-08  0.245  0.245   3.05  3.04  
...           ...    ...    ...   ...  
2022-07-18  0.645  0.655   2.66  2.65  
2022-07-19  0.640  0.645   2.65  2.65  
2022-07-20  0.635  0.645   2.67  2.67  
2022-07-21  0.630  0.635   2.67  2.67  
2022-07-22  0.630  0.630   2.65  2.67  

[3154 rows x 60 columns]

In [135]:
st.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
9CI.SI,CapitaLand Investment,1.187236,1.099263,0.305085,True,True,True
A17U.SI,Ascendas Real Estate Investment Trust,0.930607,1.427916,0.353345,False,True,False
AJBU.SI,Keppel DC Reit,1.302473,1.614026,1.102225,False,True,False
BN4.SI,Keppel Corporation,2.139215,0.420195,-0.090642,True,True,True
BS6.SI,Yangzijiang Shipbuilding Holdings [zh],19.984131,0.037573,-0.243802,True,True,True
BUOU.SI,Frasers Logistics & Commercial Trust,1.321304,1.141518,0.508292,True,True,True
C07.SI,Jardine Cycle & Carriage,1.565705,0.660959,0.036504,True,True,True
C09.SI,City Developments Limited,0.971349,0.687600,-0.328423,True,True,True
C38U.SI,CapitaLand Integrated Commercial Trust,1.881495,0.609471,0.161111,True,True,True
C52.SI,ComfortDelGro Corporation,2.602345,0.331688,-0.121951,True,True,True


In [136]:
st.top_quint

,Security
BS6.SI,Yangzijiang Shipbuilding Holdings [zh]
G13.SI,Genting Singapore PLC
O39.SI,Oversea-Chinese Banking Corp
F34.SI,Wilmar International
S68.SI,Singapore Exchange


In [137]:
time.sleep(180)

## NIFTY 50

In [138]:
nifty = OvI(nifty_df)
nifty.data

[*********************100%***********************]  50 of 50 completed


ADANIPORTS.NS             APOLLOHOSP.NS              ASIANPAINT.NS  \
                   Close        Open         Close         Open         Close   
Date                                                                            
2010-01-04    111.349998  111.000000    347.149994   327.000000    178.865005   
2010-01-05    115.190002  111.959999    366.375000   349.450012    179.429993   
2010-01-06    120.900002  115.800003    353.924988   325.625000    178.889999   
2010-01-07    119.470001  121.489998    356.924988   340.000000    177.065002   
2010-01-08    119.330002  119.400002    355.725006   359.500000    179.990005   
...                  ...         ...           ...          ...           ...   
2022-07-18    739.750000  736.950012   3947.100098  3940.100098   3017.850098   
2022-07-19    749.000000  736.950012   4020.199951  3944.399902   3019.100098   
2022-07-20    743.549988  756.950012   4110.250000  4090.000000   3006.449951   
2022-07-21    754.250000  747.700012   4150.049805  4112.000000   3066.100098   
2022-07-22    754.250000  757.650024   4125.299805  4150.200195   3067.399902   

                        AXISBANK.NS             BAJAJ-AUTO.NS               \
                   Open       Close        Open         Close         Open   
Date                                                                         
2010-01-04   179.100006  198.419998  198.779999    866.025024   882.500000   
2010-01-05   179.800003  202.559998  199.600006    870.025024   874.400024   
2010-01-06   181.475006  199.160004  204.000000    856.575012   875.000000   
2010-01-07   179.875000  200.570007  199.289993    841.549988   861.500000   
2010-01-08   177.449997  203.190002  200.600006    827.625000   842.049988   
...                 ...         ...         ...           ...          ...   
2022-07-18  2994.750000  684.500000  668.049988   3984.050049  3930.000000   
2022-07-19  2983.449951  700.599976  678.599976   4002.000000  3977.050049   
2022-07-20  3033.000000  705.700012  706.299988   4002.050049  4018.000000   
2022-07-21  2981.000000  715.549988  707.950012   4057.050049  4004.649902   
2022-07-22  3066.100098  730.750000  717.000000   4054.500000  4065.000000   

            ...     TECHM.NS                  TITAN.NS               \
            ...        Close         Open        Close         Open   
Date        ...                                                       
2010-01-04  ...   255.750000   249.199997    71.074997    71.250000   
2010-01-05  ...   259.024994   258.500000    72.532501    72.449997   
2010-01-06  ...   260.299988   258.549988    73.699997    73.550003   
2010-01-07  ...   254.574997   260.750000    75.532501    73.250000   
2010-01-08  ...   261.274994   254.500000    74.834999    76.250000   
...         ...          ...          ...          ...          ...   
2022-07-18  ...  1008.000000   985.000000  2237.449951  2196.000000   
2022-07-19  ...  1007.700012   999.950012  2241.199951  2232.500000   
2022-07-20  ...  1045.500000  1021.250000  2271.199951  2280.000000   
2022-07-21  ...  1032.750000  1025.000000  2288.600098  2285.000000   
2022-07-22  ...  1028.699951  1040.000000  2325.000000  2299.000000   

           ULTRACEMCO.NS                   UPL.NS                WIPRO.NS  \
                   Close         Open       Close        Open       Close   
Date                                                                        
2010-01-04    966.099976   914.200012  116.233330  116.666664  156.105011   
2010-01-05    986.099976   970.000000  121.933334  120.000000  158.568756   
2010-01-06    987.299988   988.000000  119.766663  122.333336  155.362503   
2010-01-07    978.349976   987.000000  118.633331  120.400002  152.606262   
2010-01-08    990.299988   994.000000  117.900002  118.666664  150.311264   
...                  ...          ...         ...         ...         ...   
2022-07-18   5982.399902  5832.000000  685.049988  676.950012  404.750000   
2022-07-19   6085.

In [139]:
nifty.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
ADANIPORTS.NS,Adani Ports,175.751692,0.037779,5.795045,True,True,True
APOLLOHOSP.NS,Apollo Hospitals,199.635516,0.061423,11.615596,True,True,True
ASIANPAINT.NS,Asian Paints,64.221207,0.263452,16.126744,True,True,True
AXISBANK.NS,Axis Bank,43.055363,0.085441,2.676175,True,True,True
BAJAJ-AUTO.NS,Bajaj Auto,30.154364,0.148287,3.594334,True,True,True
BAJAJFINSV.NS,Bajaj Finserv,161.466873,0.231309,36.149558,True,True,True
BAJFINANCE.NS,Bajaj Finance,111.910354,1.745150,192.606964,True,False,False
BHARTIARTL.NS,Bharti Airtel,51.585014,0.043157,1.241456,True,True,True
BPCL.NS,Bharat Petroleum,304.604463,0.009658,2.023837,True,True,True
BRITANNIA.NS,Britannia Industries,134.062300,0.170833,22.196226,True,True,True


In [140]:
nifty.top_quint

,Security
TATACONSUM.NS,Tata Consumer Products
CIPLA.NS,Cipla
ONGC.NS,Oil and Natural Gas Corporation
UPL.NS,United Phosphorus Limited
EICHERMOT.NS,Eicher Motors
BPCL.NS,Bharat Petroleum
DIVISLAB.NS,Divi's Laboratories
SBIN.NS,State Bank of India
NESTLEIND.NS,Nestlé India
TATAMOTORS.NS,Tata Motors


In [141]:
time.sleep(180)

## BSE SENSEX

In [142]:
bse = OvI(bse_df)
bse.data

[*********************100%***********************]  30 of 30 completed


ASIANPAINT.BO              AXISBANK.BO             BAJAJ-AUTO.BO  \
                   Close         Open       Close        Open         Close   
Date                                                                          
2010-01-04    178.964996   173.800003  198.610001  199.800003    866.200012   
2010-01-05    179.039993   179.100006  202.000000  200.199997    868.250000   
2010-01-06    178.039993   180.679993  199.059998  203.330002    856.525024   
2010-01-07    177.649994   178.000000  200.589996  200.000000    841.775024   
2010-01-08    179.570007   173.000000  202.710007  201.800003    827.349976   
...                  ...          ...         ...         ...           ...   
2022-07-18   3019.199951  2990.000000  684.450012  662.250000   3985.199951   
2022-07-19   3018.050049  2985.000000  700.549988  680.000000   4002.250000   
2022-07-20   3011.750000  3020.050049  705.049988  708.700012   4001.649902   
2022-07-21   3069.300049  2987.899902  715.650024  706.150024   4057.449951   
2022-07-22   3066.149902  3070.000000  730.950012  716.599976   4054.800049   

                        BAJAJFINSV.BO               BAJFINANCE.BO  \
                   Open         Close          Open         Close   
Date                                                                
2010-01-04   885.000000    353.194916    342.926758     33.180489   
2010-01-05   877.000000    353.587952    358.648346     33.000767   
2010-01-06   867.599976    344.548065    354.717957     33.151344   
2010-01-07   860.000000    342.386322    346.365875     33.437927   
2010-01-08   848.500000    344.056763    346.267609     34.108238   
...                 ...           ...           ...           ...   
2022-07-18  3920.649902  12246.650391  11899.000000   6056.450195   
2022-07-19  3986.000000  12445.299805  12200.000000   6062.500000   
2022-07-20  4007.000000  12417.299805  12650.000000   6070.299805   
2022-07-21  4000.000000  12718.250000  12400.000000   6268.799805   
2022-07-22  4068.000000  12741.049805  12914.200195   6256.299805   

                         ... SUNPHARMA.BO                   TCS.BO  \
                   Open  ...        Close        Open        Close   
Date                     ...                                         
2010-01-04    32.320740  ...   150.809998  153.800003   375.850006   
2010-01-05    33.894520  ...   155.375000  152.899994   375.450012   
2010-01-06    33.505932  ...   156.869995  156.110001   366.899994   
2010-01-07    33.233917  ...   154.199997  156.000000   357.325012   
2010-01-08    33.505932  ...   157.455002  154.509995   350.250000   
...                 ...  ...          ...         ...          ...   
2022-07-18  5952.000000  ...   882.750000  890.000000  3060.350098   
2022-07-19  6040.000000  ...   874.900024  880.000000  3075.100098   
2022-07-20  6150.000000  ...   867.250000  882.049988  3163.949951   
2022-07-21  6075.000000  ...   869.400024  875.900024  3177.699951   
2022-07-22  6270.000000  ...   874.599976  869.500000  3171.000000   

                            TECHM.BO                  TITAN.BO               \
                   Open        Close         Open        Close         Open   
Date                                                                          
2010-01-04   375.350006   255.899994   249.500000    71.037498    71.000000   
2010-01-05   378.125000   259.137512   258.750000    72.527496    71.900002   
2010-01-06   376.450012   260.174988   260.000000    73.709999    73.000000   
2010-01-07   366.500000   254.524994   260.250000    75.489998    73.650002   
2010-01-08   350.000000   261.424988   255.000000    74.757500    73.250000   
...                 ...          ...          ...          ...          ...   
2022-07-18  3023.399902  1008.700012   980.000000  2236.850098  2190.000000   
2022-07-19  3026.000000  1007.599976  1002.000000  2239.649902  2235.000000   
2022-07-20  3109.449951  1046.300049  1019.000000  2270.800049  2276.000000   
2022-07-

In [143]:
bse.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
ASIANPAINT.BO,Asian Paints,281.717787,0.062622,16.641829,True,True,True
AXISBANK.BO,Axis Bank,213.921449,0.017102,2.658408,True,True,True
BAJAJ-AUTO.BO,Bajaj Auto,104.342872,0.043910,3.581695,True,True,True
BAJAJFINSV.BO,Bajaj Finserv,2785.857936,0.013337,36.153851,True,True,True
BAJFINANCE.BO,Bajaj Finance,1629.474191,0.118792,192.569202,True,True,True
BHARTIARTL.BO,Bharti Airtel,304.884389,0.007346,1.239718,True,True,True
DRREDDY.BO,Dr Reddy's Laboratories,203.511066,0.018855,2.837281,True,True,True
HCLTECH.BO,HCL Technologies,346.754546,0.028529,8.892473,True,True,True
HDFC.BO,HDFC,91.543869,0.046786,3.283010,True,True,True
HDFCBANK.BO,HDFC Bank,114.399436,0.072033,7.240497,True,True,True


In [144]:
bse.top_quint

,Security
ONGC.BO,Oil and Natural Gas Corporation
BAJAJFINSV.BO,Bajaj Finserv
SUNPHARMA.BO,Sun Pharma
BAJFINANCE.BO,Bajaj Finance
M&M.BO,Mahindra & Mahindra
TITAN.BO,Titan Company


In [145]:
time.sleep(180)

## MSCI Brazil

In [146]:
brz = OvI(brz_df)
brz.data

[*********************100%***********************]  48 of 48 completed

1 Failed download:
- BBDC3: No data found, symbol may be delisted


ABEV3.SA              AMER3.SA            B3SA3.SA         \
                Close       Open      Close       Open    Close   Open   
Date                                                                     
2010-01-04   6.113138   5.932941  41.075191  41.949505    12.95  12.29   
2010-01-05   6.149098   6.073583  40.094894  41.507931    13.00  12.97   
2010-01-06   6.211828   6.099554  40.536469  40.174377    13.49  12.93   
2010-01-07   6.240995   6.221017  40.995708  40.183212    13.81  13.30   
2010-01-08   6.229008   6.193049  40.015411  41.331303    13.49  13.79   
...               ...        ...        ...        ...      ...    ...   
2022-07-18  14.490000  14.600000  15.740000  16.480000    10.27  10.09   
2022-07-19  14.480000  14.490000  16.190001  15.700000    10.21  10.27   
2022-07-20  14.300000  14.410000  16.920000  16.280001    10.54  10.19   
2022-07-21  14.560000  14.240000  16.930000  16.889999    11.01  10.56   
2022-07-22  14.510000  14.540000  15.930000  17.000000    10.72  11.08   

             BBAS3.SA              BBDC3.SA             ... UGPA3.SA           \
                Close       Open      Close       Open  ...    Close     Open   
Date                                                    ...                     
2010-01-04  29.900000  29.799999   9.076981   8.781247  ...   7.3125   7.3125   
2010-01-05  29.600000  30.100000   8.948147   9.050629  ...   7.3125   7.3125   
2010-01-06  29.639999  29.490000   8.895441   8.901298  ...   7.3125   7.3125   
2010-01-07  29.650000  29.440001   8.874945   8.839808  ...   7.3125   7.3125   
2010-01-08  29.820000  29.700001   8.886657   8.872017  ...   7.3125   7.3125   
...               ...        ...        ...        ...  ...      ...      ...   
2022-07-18  33.750000  33.500000  13.860000  13.830000  ...  12.0200  12.0000   
2022-07-19  34.660000  33.880001  14.330000  13.900000  ...  12.0100  12.0000   
2022-07-20  34.669998  34.540001  14.100000  14.260000  ...  12.7200  11.9600   
2022-07-21  34.910000  34.619999  14.270000  14.030000  ...  12.6800  12.6300   
2022-07-22  34.669998  34.919998  14.160000  14.320000  ...  12.4600  12.6100   

             VALE3.SA              VBBR3.SA              VIVT3.SA             \
                Close       Open      Close       Open      Close       Open   
Date                                                                           
2010-01-04  51.490002  50.189999        NaN        NaN  37.779999  37.900002   
2010-01-05  51.970001  51.540001        NaN        NaN  37.639999  37.799999   
2010-01-06  53.070000  52.000000        NaN        NaN  37.000000  37.599998   
2010-01-07  53.290001  52.509998        NaN        NaN  37.000000  36.799999   
2010-01-08  53.810001  53.299999        NaN        NaN  36.970001  36.840000   
...               ...        ...        ...        ...        ...        ...   
2022-07-18  68.730003  70.389999  16.430000  17.010000  46.910000  47.169998   
2022-07-19  68.879997  68.209999  16.129999  16.510000  47.509998  47.060001   
2022-07-20  67.389999  68.459999  17.020000  16.020000  46.889999  47.160000   
2022-07-21  68.570000  67.309998  16.980000  16.790001  46.980000  47.080002   
2022-07-22  69.209999  69.800003  16.540001  16.980000  46.830002  46.990002   

             WEGE3.SA             
                Close       Open  
Date                              
2010-01-04   2.711538   2.735207  
2010-01-05   2.705621   2.711538  
2010-01-06   2.720414   2.696745  
2010-01-07   2.766272   2.742603  
2010-01-08   2.866863   2.758875  
...               ...        ...  
2022-07-18  26.120001  26.900000  
2022-07-19  26.959999  26.250000  
2022-07-20  25.990000  27.000000  
2022-07-21  25.330000  26.049999  
2022-07-22  25.840000  25.490000  

[3118 rows x 94 columns]

In [147]:
brz.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
ABEV3.SA,AMBEV SA,0.586553,4.393895,1.445667,False,False,False
AMER3.SA,AMERICANAS SA,21.150684,0.018005,-0.620258,True,True,True
B3SA3.SA,B3 BRASIL BOLSA BALCAO SA,0.336836,2.550856,-0.127746,False,True,False
BBAS3.SA,BANCO DO BRASIL SA,5.674574,0.206372,0.163423,True,True,True
BBDC3.SA,CPFL ENERGIA SA,0.519653,3.112630,0.612527,False,False,False
BBDC4.SA,BANCO BRADESCO PREF SA,2.920355,0.446893,0.303744,True,True,True
BBSE3.SA,BB SEGURIDADE SA,0.949345,1.772315,0.644838,False,True,False
BPAC11.SA,BCO BTG PACTUAL UNT SA,5.592824,0.857774,3.938461,True,True,True
BRFS3.SA,BRF BRASIL FOODS SA,2.696007,0.276903,-0.288938,True,True,True
CCRO3.SA,COMPANHIA CONCESSOES RODOVIARIAS S,1.611006,0.744847,0.228307,True,True,True


In [148]:
brz.top_quint

,Security
PRIO3.SA,PETRO RIO SA
CSNA3.SA,COMPANHIA SIDERURGICA NACIONAL SA
MGLU3.SA,MAGAZINE LUIZA SA
GGBR4.SA,GERDAU PREF SA
ELET3.SA,CENTRAIS ELETR BRAS-ELETROBRAS
PETR4.SA,PETROLEO BRASILEIRO PREF SA


In [149]:
time.sleep(180)

## TA 125

In [150]:
ta = OvI(ta_df)
ta.data

[*********************100%***********************]  126 of 126 completed

14 Failed downloads:
- IFF.TA: No data found, symbol may be delisted
- RATI.L.TA: No data found, symbol may be delisted
- ASPR.TA: No data found, symbol may be delisted
- HAP.TA: No data found, symbol may be delisted
- GZT.TA: No data found, symbol may be delisted
- HAML.TA: No data found, symbol may be delisted
- ARKO.TA: No data found, symbol may be delisted
- DEDR.L.TA: No data found, symbol may be delisted
- FRSM.TA: No data found, symbol may be delisted
- PRGO.TA: No data found, symbol may be delisted
- ISRA.L.TA: No data found, symbol may be delisted
- ASPF.TA: No data found, symbol may be delisted
- ITMR.TA: No data found, symbol may be delisted
- BYSD.TA: No data found, symbol may be delisted


ADGR.TA                   AFPR.TA                AFRE.TA  \
                 Close        Open         Close          Open    Close   
Date                                                                      
2010-01-03  417.799988  386.000000   5422.534180   5422.534180   6649.0   
2010-01-04  444.500000  425.000000   5633.588379   5633.588379   6685.0   
2010-01-05  456.600006  445.399994   5775.904297   5775.904297   6797.0   
2010-01-06  453.299988  458.000000   5807.853027   5807.853027   6790.0   
2010-01-07  431.000000  440.399994   5842.705566   5842.705566   6782.0   
...                ...         ...           ...           ...      ...   
2022-07-17  700.799988  685.700012  16800.000000  16760.000000  17200.0   
2022-07-18  706.000000  700.799988  16950.000000  16800.000000  18060.0   
2022-07-19  706.200012  706.000000  17010.000000  16950.000000  18180.0   
2022-07-20  707.799988  706.200012  16960.000000  17010.000000  18330.0   
2022-07-21  702.099976  707.799988  17170.000000  16960.000000  18680.0   

                    ALHE.TA         ALLT.TA          ...  TEVA.TA           \
               Open   Close    Open   Close    Open  ...    Close     Open   
Date                                                 ...                     
2010-01-03   6481.0  1527.0  1455.0     NaN     NaN  ...  21500.0  21400.0   
2010-01-04   6770.0  1582.0  1528.0     NaN     NaN  ...  21470.0  21550.0   
2010-01-05   6685.0  1521.0  1597.0     NaN     NaN  ...  21840.0  21750.0   
2010-01-06   6605.0  1530.0  1534.0     NaN     NaN  ...  21490.0  21420.0   
2010-01-07   6831.0  1500.0  1530.0     NaN     NaN  ...  21460.0  21400.0   
...             ...     ...     ...     ...     ...  ...      ...      ...   
2022-07-17  16990.0  4735.0  4770.0  1722.0  1705.0  ...   2464.0   2440.0   
2022-07-18  17200.0  4700.0  4799.0  1727.0  1722.0  ...   2476.0   2469.0   
2022-07-19  18060.0  4780.0  4700.0  1724.0  1727.0  ...   2488.0   2460.0   
2022-07-20  18940.0  4880.0  4875.0  1749.0  1724.0  ...   2549.0   2538.0   
2022-07-21  18330.0  4954.0  4857.0  1830.0  1800.0  ...   2475.0   2515.0   

                 TSEM.TA               VCTR.TA         VTNA.TA          \
                   Close          Open   Close    Open   Close    Open   
Date                                                                     
2010-01-03   1052.820557   1052.820557     NaN     NaN     NaN     NaN   
2010-01-04   1052.820557   1052.820557     NaN     NaN     NaN     NaN   
2010-01-05   1052.820557   1052.820557     NaN     NaN     NaN     NaN   
2010-01-06   1052.820557   1052.820557     NaN     NaN     NaN     NaN   
2010-01-07   1052.820557   1052.820557     NaN     NaN     NaN     NaN   
...                  ...           ...     ...     ...     ...     ...   
2022-07-17  15950.000000  16000.000000  4754.0  4778.0  2558.0  2497.0   
2022-07-18  15840.000000  15920.000000  4818.0  4754.0  2557.0  2558.0   
2022-07-19  15790.000000  15840.000000  4820.0  4820.0  2555.0  2557.0   
2022-07-20  15870.000000  15840.000000  4830.0  4820.0  2542.0  2555.0   
2022-07-21  15990.000000  15950.000000  4865.0  4830.0  2611.0  2542.0   

            YHNF.TA           
              Close     Open  
Date                          
2010-01-03      NaN      NaN  
2010-01-04      NaN      NaN  
2010-01-05      NaN      NaN  
2010-01-06      NaN      NaN  
2010-01-07      NaN      NaN  
...             ...      ...  
2022-07-17  21260.0  21440.0  
2022-07-18  21130.0  21260.0  
2022-07-19  21130.0  21130.0  
2022-07-20  21080.0  21130.0  
2022-07-21  21600.0  21080.0  

[3106 rows x 224 columns]

In [151]:
ta.together

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
ADGR.TA,Adgar Investment & Development Ltd.,1.151658,1.590987,0.818912,False,True,False
AFPR.TA,AFI Properties Ltd.,0.149699,20.486571,2.166416,False,False,False
AFRE.TA,Africa Israel Residences Ltd.,3.225904,0.874544,1.882271,True,True,True
ALHE.TA,Alony Hetz Properties & Investments Ltd.,3.539504,0.948846,2.404811,True,True,True
ALLT.TA,Allot Ltd.,1.303505,0.358270,-0.513298,True,True,True
...,...,...,...,...,...,...,...
TEVA.TA,Teva Pharmaceutical Industries Ltd.,0.182406,0.600554,-0.884346,False,True,False
TSEM.TA,Tower Semiconductor Ltd.,14.613084,0.975789,14.187773,True,True,True
VCTR.TA,Victory Supermarket Chain Ltd.,0.239468,0.113224,-0.972669,True,True,True
VTNA.TA,Vitania Ltd.,2.253564,1.033145,1.327094,True,True,True


In [152]:
ta.top_quint

,Security
ILDC.TA,The Israel Land Development Co. Ltd.
BVC.TA,BATM Advanced Communications Ltd.
FORTY.TA,Formula Systems Ltd.
ELWS.TA,Elecreaon Wireless Ltd.
DELT.TA,Delta-Galil Industries Ltd.
PHOE.TA,The Phoenix Holdings Ltd.
SMT.TA,Summit Real Estate Holdings Ltd.
CGEN.TA,Compugen Ltd.
NFTA.TA,Naphtha Israel Petroleum Corp. Ltd.
ENLT.TA,Enlight Renewable Energy Ltd.


# Top Overnight Returners

In [154]:
top_quints = [sp.top_quint, 
              daq.top_quint,
              tsx.top_quint,
              mex.top_quint,
              asx.top_quint,
              obx.top_quint,
              dax.top_quint,
              aex.top_quint,
              cac.top_quint,
              ftse.top_quint,
              mib.top_quint,
              ibex.top_quint,
              nikkei.top_quint,
              kospi.top_quint,
              tsec.top_quint,
              hs.top_quint,
              csi.top_quint,
              st.top_quint,
              nifty.top_quint,
              bse.top_quint,
              brz.top_quint,
              ta.top_quint]

In [155]:
top_df = pd.concat(top_quints)
top_df

,Security
ENPH,Enphase
TSLA,"Tesla, Inc."
AAL,American Airlines Group
MU,Micron Technology
MGM,MGM Resorts
...,...
IES.TA,IES Holdings Ltd.
PLSN.TA,Plasson Industries Ltd.
DLEKG.TA,Delek Group Ltd.
ECP.TA,Electra Consumer Products Ltd.


In [156]:
with open('Overnight Tickers.txt', "wb") as file:
    pickle.dump(top_df, file)